#### Vorgehen gemäß Paper
Poljak, Željko. 2024. „From Speech to Feed: How Parliamentary Debates Shape Party Agendas on Social Media‌“. Swiss Political Science Review: spsr.12634. doi:10.1111/spsr.12634.

Ziel ist es eine logistische Regression mit den vorhandenen Daten unter Berücksichtigung der relativen Aufmerksamkeit eines Themas durchzuführen. Vllt nochmal mit mehr Lags untersuchen, als nur einem!

##### Daten Preprocessing

In [2]:
import pandas as pd
subset_reden = pd.read_csv("subset_reden.csv")
subset_posts = pd.read_csv("subset_posts.csv")

#### Fehlerhafte Topics
Ermitteln der fehlerhaft deklarierten Themen und Umbennenung dieser

##### Zielvariable
In der bisherigen Analyse hatte ich einen zentralen Denkfehler: Da die Werte in der Zeitreihe nicht konstant sind, muss ich als Zielvariable die Facebookposts am Tag *nach* einer Bundestagswahl analysieren. Bisher hatte ich posts_common, also die Facebookposts an Tag y als Zielvariable und über shift(-1) die Redebeiträge -1 gelaggt. Dieses Vorgehen geht, wie bei der Zeitreihenanalyse, *nicht*, da die Abstände zwischen den einzelnen Punkten in den Daten nicht gleich verteilt sind, da nicht an jedem Tag Sitzungen sind. Stattdessen wähle ich jetzt einen Tag in die Zukunft gelaggte Facebookposts als Zielvariable. Somit kann ich die Sitzungsdaten so belassen wie sie sind und einfach t+1 gelaggte Werte der Facebookdaten als Zielvariable nutzen!

In [3]:
subset_reden["date"] = pd.to_datetime(subset_reden["date"])
subset_posts["date"] = pd.to_datetime(subset_posts["date"])
# 🔹 2️⃣ Gruppiere die Themen täglich und zähle die Häufigkeit der Themen pro Tag
redethemen_täglich = subset_reden.groupby(['date','Topic']).size().unstack(fill_value=0)
postthemen_täglich = subset_posts.groupby(['date', 'Topic' ]).size().unstack(fill_value=0)
# Berechnung der täglichen Summen der Wortanzahlen
reden_komplexität_täglich = subset_reden.groupby('date')['komplexität'].sum()
posts_komplexität_täglich = subset_posts.groupby('date')['komplexität'].sum()


# Führe einen inner join durch, damit nur gemeinsame Tage beibehalten werden
common_dates = redethemen_täglich.index.intersection(postthemen_täglich.index)
redethemen_täglich_aligned = redethemen_täglich.loc[common_dates]
postthemen_täglich_aligned = postthemen_täglich.loc[common_dates]
rede_komplex = reden_komplexität_täglich.loc[common_dates]
posts_komplex = posts_komplexität_täglich.loc[common_dates]
# Zielvariable: Facebookposts um einen Tag in die Zukunft gelaggt
# posts_shifted enthält die Facebookposts mit lag t+1, Die Daten hier sind zwar die gleichen wie in den anderen Dataframes,
# # die Werte jedoch die vom Vortag, weshalb die Analyse zulässig ist
# posts_shifted=postthemen_täglich_aligned.shift().dropna()
# # # Erneute Anpassung des Datums
# common_dates2 = redethemen_täglich.index.intersection(posts_shifted.index)
# redethemen_täglich_aligned = redethemen_täglich.loc[common_dates2]
# postthemen_täglich_aligned = postthemen_täglich.loc[common_dates2]


In [4]:
posts_komplex

date
2022-05-31    2712
2022-06-01    4939
2022-06-02    2277
2022-06-03    4780
2022-06-22    3475
              ... 
2023-04-27    2553
2023-04-28    2364
2023-05-10    2350
2023-05-11    3783
2023-05-12    4190
Name: komplexität, Length: 66, dtype: int64

In [5]:
# Nur Topics berücksichtigen, die in beiden Datensätzen vorkommen
# Filterfunktion
def filter_common_topics(df1, df2):
    """
    Filtert gemeinsame Topics aus zwei DataFrames und gibt neue DataFrames zurück,
    die nur die gemeinsamen Topics enthalten.
    
    Args:
    - df1: Erster DataFrame
    - df2: Zweiter DataFrame
    
    Returns:
    - filtered_df1: DataFrame mit den gemeinsamen Topics aus df1
    - filtered_df2: DataFrame mit den gemeinsamen Topics aus df2
    """
    # Extrahiere die Spaltennamen, die Topics darstellen
    topics_df1 = set(df1.columns)
    topics_df2 = set(df2.columns)
    
    # Finde gemeinsame Topics
    common_topics = topics_df1.intersection(topics_df2)
    
    print(f"Gemeinsame Topics: {common_topics}")
    
    # Filtere DataFrames auf die gemeinsamen Topics
    filtered_df1 = df1[list(common_topics)]
    filtered_df2 = df2[list(common_topics)]
    
    return filtered_df1, filtered_df2

In [6]:
rede_common, post_common = filter_common_topics(redethemen_täglich_aligned, postthemen_täglich_aligned)
#posts_shifted_common, post_common = filter_common_topics(post_common, posts_shifted)

Gemeinsame Topics: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 63, 64, 65, 67, 68, 69, 70, 72, 74, 75, 77, 78, 79, 80, 81, 83, 84, 85, 88, 90, 91, 92, 93}


In [7]:
post_common

Topic,0,1,2,3,4,5,6,7,8,9,...,80,81,83,84,85,88,90,91,92,93
date,,,,,,,,,,,,,,,,,,,,,
2022-05-31,0,3,3,3,4,0,3,2,0,2,...,0,0,0,0,0,0,0,0,0,0
2022-06-01,1,10,6,0,7,0,18,0,5,0,...,0,0,0,0,0,0,0,0,0,0
2022-06-02,3,6,4,3,1,0,0,0,3,2,...,0,1,0,0,0,0,0,0,0,0
2022-06-03,7,4,8,2,4,0,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2022-06-22,7,2,5,2,1,0,2,3,0,3,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-27,6,2,1,3,4,0,1,0,0,7,...,0,1,0,0,0,0,0,0,0,0
2023-04-28,1,8,1,8,2,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
2023-05-10,2,5,2,8,1,0,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
reden_relativ =  rede_common.div(rede_common.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
post_relativ = post_common.div(post_common.sum(axis=1), axis=0)

In [9]:
# Überprüfen, ob es gemeinsame Spalten mit nur 0-Werten gibt.
# Prozentsatz der Nullwerte je Spalte berechnen
percent_zeros_reden = (post_relativ == 0).mean() * 100


# Spalten ausgeben, die 100 % Nullwerte haben
print("Spalten mit 100% Nullwerten:", percent_zeros_reden[percent_zeros_reden > 90].index.tolist())

Spalten mit 100% Nullwerten: [5, 31, 44, 52, 55, 59, 60, 61, 63, 64, 69, 72, 74, 77, 78, 80, 83, 85, 90, 91, 92, 93]


In [10]:
# Überprüfen, wie viele Themen es gibt, die zu mehr als 90% aus 0-Werten bestehen
# Überprüfen, ob es gemeinsame Spalten mit nur 0-Werten gibt.
# Prozentsatz der Nullwerte je Spalte berechnen
percent_zeros_reden = (reden_relativ == 0).mean() * 100
percent_zeros_posts = (post_relativ == 0).mean() * 100

# Spalten ausgeben, die 100 % Nullwerte haben
print("Themen mit mehr als 90% Nullwerten; Reden:", percent_zeros_reden[percent_zeros_reden > 90].index.tolist())
print("Themen mit mehr als 90% Nullwerten; Posts:", percent_zeros_posts[percent_zeros_posts > 90].index.tolist())
def remove_near_constant(df):
    """
    Entfernt Topics, wo mehr als 90% der Werte 0 sind, da diese als Konstant interpretiert werden
    """
    percent_zeros_reden = (df == 0).mean() * 100
    df = df.loc[:, percent_zeros_reden <= 90]
    return df
reden_relativ = remove_near_constant(reden_relativ)
post_relativ = remove_near_constant(post_relativ)
rede_relativ_reduced, post_relativ_reduced = filter_common_topics(reden_relativ, post_relativ)

Themen mit mehr als 90% Nullwerten; Reden: [9, 26, 27, 28, 32, 33, 35, 38, 39, 44, 45, 46, 47, 51, 53, 55, 56, 58, 59, 60, 63, 65, 67, 70, 79, 81, 83, 84, 88, 91, 93]
Themen mit mehr als 90% Nullwerten; Posts: [5, 31, 44, 52, 55, 59, 60, 61, 63, 64, 69, 72, 74, 77, 78, 80, 83, 85, 90, 91, 92, 93]
Gemeinsame Topics: {0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 29, 30, 34, 36, 37, 40, 41, 43, 54, 68, 75}


#### Logisitsche Regression

In [11]:
def logistic_regression_with_lags_SM(relativ_rede, relativ_posts, post_to_shift):
    """
    Führt eine logistische Regression mit zeitversetzten unabhängigen Variablen durch.
    
    Args:
    - relativ_rede: DataFrame mit Erwähnungen im Parlament; relative Anteile.
    - relativ_posts: DataFrame mit Erwähnungen auf Social Media; relative Anteile.
    """
    import statsmodels.api as sm
    
    # # Erneute Anpassung des Datums
    # common_dates2 = redethemen_täglich.index.intersection(posts_shifted.index)
    # redethemen_täglich_aligned = redethemen_täglich.loc[common_dates2]
    # postthemen_täglich_aligned = postthemen_täglich.loc[common_dates2]
    # Zielvariable (t+1): Thema auf Social Media erwähnt, einen Tag nach Besprechung im Parlament
    posts_shifted_common = post_to_shift.shift().dropna()
    common_dates2 = relativ_rede.index.intersection(posts_shifted_common.index)
    relativ_rede = relativ_rede.loc[common_dates2]
    relativ_posts=relativ_posts.loc[common_dates2]
    y = (posts_shifted_common > 0).astype(int).stack()

    # Unabhängige Variablen (t-1): Erwähnungen im Parlament mit Zeitversatz
    #absolute = rede_common.shift(lag_days).stack()
    
    relative_reden = relativ_rede.stack()

    relative_posts = relativ_posts.stack()

    # DataFrame für die logistische Regression
    X = pd.DataFrame({ 'posts_relative': relative_posts, 'reden_relativ': relative_reden}).dropna()

    # Zielvariable und unabhängige Variablen ausrichten
    y = y[X.index]

    # Logistische Regression
    X = sm.add_constant(X)
    model = sm.Logit(y, X).fit()
    print(model.summary())

In [12]:
# Model 1: Zielvariable: Thema auf Social Media erwähnt (t+1) in Abhängigkeit von relativem Anteil an Erwähnungen am Vortag im Bundestag und auf Social Media
logistic_regression_with_lags_SM(reden_relativ, post_relativ, post_common)


Optimization terminated successfully.
         Current function value: 0.598824
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2272
Method:                           MLE   Df Model:                            2
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.1356
Time:                        11:03:14   Log-Likelihood:                -1362.3
converged:                       True   LL-Null:                       -1576.1
Covariance Type:            nonrobust   LLR p-value:                 1.452e-93
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.7187      0.057    -12.669      0.000      -0.830      -0.608
posts_relative   

#### Logistische Regression mit Fixed Effects
Folgendes Vorgehen scheint nun zu funktionieren! Topic 0 bei den Fixed Effects zur Referenzkategorie zu machen, war hier der Knackpunkt, was wahrscheinlich daran liegt, dass dieses Topic am meisten erwähnt wird. Auch das rauslöschen der quasi "leeren" Topic mit mehr als 90% 0-Werten scheint dem Modell sehr gut getan zu haben. In dem vorhandenen Code wird Thema 0 gelöscht, um perfekte Multikolinearität zu vermeiden. Die Konstante entspricht dann in diesem Modell Topic 0 und alle anderen Topics sind im Verhältnis zu Topic 0 zu interpretieren.

In [13]:
# Mit fixed_effects
def logistic_regression_with_lags_SM_fixed_effects(relativ_rede, relativ_posts, post_to_shift):
    """
    Führt eine logistische Regression mit Fixed Effects für Themen durch.
    
    Args:
    - relativ_rede: DataFrame mit Erwähnungen im Parlament; relative Anteile.
    - relativ_posts: DataFrame mit Erwähnungen auf Social Media; relative Anteile.
    """
    import statsmodels.api as sm

    # Zielvariable (t+1): Thema auf Social Media erwähnt, einen Tag nach Besprechung im Parlament
    posts_shifted_common = post_to_shift.shift().dropna()
    common_dates2 = relativ_rede.index.intersection(posts_shifted_common.index)
    relativ_rede = relativ_rede.loc[common_dates2]
    relativ_posts=relativ_posts.loc[common_dates2]
    y = (posts_shifted_common > 0).astype(int).stack()

    # Unabhängige Variablen: relative Anteile im Parlament und auf Social Media
    relative_reden = relativ_rede.stack()
    relative_posts = relativ_posts.stack()

    # Zielvariable und unabhängige Variablen als DataFrame zusammenführen
    X = pd.DataFrame({'posts_relative': relative_posts, 'reden_relativ': relative_reden}).dropna()
    y = y.reindex(X.index).dropna()

    # Konvertiere MultiIndex zu flachem Index, um die Kompatibilität sicherzustellen
    X.index = X.index.to_flat_index()
    y.index = y.index.to_flat_index()

    # Dummy-Variablen für Themen (Fixed Effects)
    topics = [idx[1] for idx in X.index]  # Der zweite Wert im Tupel repräsentiert das Topic
    topic_dummies = pd.get_dummies(topics, prefix='topic',drop_first=True).astype(int) # ohne.astype(int) kommen hier Fehlermeldungen
    # Versuch aus den Topics Topic 0 zu löschen und somit bessere Daten zu erlangen

    # Füge Dummy-Variablen hinzu
    X = pd.concat([X.reset_index(drop=True), topic_dummies.reset_index(drop=True)], axis=1)

    # Logistische Regression
    X = sm.add_constant(X)
    model = sm.Logit(y.values, X).fit(maxiter=100)
    print(model.summary())



#### Logistische Regression mit Social Media Nutzung als Kontrollvariable
Zusätzlich noch die Aufnahme der Landtagswahlen in Schleswig-Holstein (08-05-2022), Nordrhein-Westfalen (15-05-2022) und Niedersachsen (09-10-2022). Angenommen wird, dass diese die Aktivität auf Facebook bis zu 30 Tage nach der Wahl beschäftigen können *(--> prüfen, ob es hier Quellen für gibt!)*. Die Dummy-Variablen sind dementsprechend 0/1 codiert; 1 für den Zeitraum Wahl+30 Tage und 0 für die restliche Zeit

In [14]:
import pandas as pd
import numpy as np

# Liste der Landtagswahltermine
landtagswahltermine = [
    "2022-05-08",  # Schleswig-Holstein
    "2022-05-15",  # Nordrhein-Westfalen
    "2022-10-09"   # Niedersachsen
]

# Konvertiere die Termine zu Datetime
landtagswahltermine = pd.to_datetime(landtagswahltermine)

# Zeitfenster: 30 Tage vor und nach der Wahl
zeitfenster = 30

# Erstelle eine vollständige Datumsreihe im Beobachtungszeitraum
beobachtungszeitraum = pd.date_range(start="2022-05-01", end="2023-05-31", freq="D")

# Initialisiere die Series mit Nullen
landtagswahlen_series = pd.Series(0, index=beobachtungszeitraum)

# Inkrementiere die Werte für sich überlappende Zeitfenster
for wahltermin in landtagswahltermine:
    startdatum = wahltermin - pd.Timedelta(days=zeitfenster)
    enddatum = wahltermin + pd.Timedelta(days=zeitfenster)
    landtagswahlen_series.loc[startdatum:enddatum] += 1

# Kontrolliere die Series
landtagswahlen_series


2022-05-01    2
2022-05-02    2
2022-05-03    2
2022-05-04    2
2022-05-05    2
             ..
2023-05-27    0
2023-05-28    0
2023-05-29    0
2023-05-30    0
2023-05-31    0
Freq: D, Length: 396, dtype: int64

In [15]:
# Annahme: subset_posts enthält eine 'date'-Spalte und repräsentiert alle Social Media Posts
# Gruppieren der Anzahl der Posts nach Datum
social_media_usage = subset_posts.groupby('date').size()
social_media_usage


date
2022-05-24     8
2022-05-25    37
2022-05-26    15
2022-05-27    29
2022-05-28    25
              ..
2023-05-17    68
2023-05-18     7
2023-05-19    22
2023-05-20    17
2023-05-21    10
Length: 363, dtype: int64

In [16]:
# Logistische Regression mit der Kontrollvariable 'social_media_usage'
def logistic_regression_with_control(relativ_rede, relativ_posts, post_to_shift, social_media_usage):
    """
    Führt eine logistische Regression mit der Kontrollvariable 'social_media_usage' durch.
    
    Args:
    - relativ_rede: DataFrame mit relativen Anteilen der Themen im Bundestag.
    - relativ_posts: DataFrame mit relativen Anteilen der Themen auf Social Media.
    - post_to_shift: DataFrame mit Themen auf Social Media (Zielvariable, t+1).
    - social_media_usage: Series mit täglicher Social-Media-Aktivität.
    """
    import statsmodels.api as sm

    # Zielvariable (t+1): Thema auf Social Media erwähnt, einen Tag nach Besprechung im Parlament
    posts_shifted_common = post_to_shift.shift().dropna()
    common_dates2 = relativ_rede.index.intersection(posts_shifted_common.index)
    relativ_rede = relativ_rede.loc[common_dates2]
    relativ_posts = relativ_posts.loc[common_dates2]
    social_media_usage = social_media_usage.loc[common_dates2]
    landtagswahlen = landtagswahlen_series.loc[common_dates2]

    y = (posts_shifted_common > 0).astype(int).stack()

    # Unabhängige Variablen
    relative_reden = relativ_rede.stack()
    relative_posts = relativ_posts.stack()

    # Kontrollvariablen hinzufügen
    social_media_usage_stacked = social_media_usage.reindex(relativ_rede.index)
    social_media_usage_stacked = social_media_usage_stacked.repeat(relativ_rede.shape[1])
    social_media_usage_stacked.index = relative_reden.index

   # Dummy-Variable für Landtagswahlen
    landtagswahlen_stacked = landtagswahlen.reindex(relativ_rede.index)
    landtagswahlen_stacked = landtagswahlen_stacked.repeat(relativ_rede.shape[1])
    landtagswahlen_stacked.index = relative_reden.index

   
    
    # DataFrame für die logistische Regression
    X = pd.DataFrame({
        'posts_relative': relative_posts, 
        'reden_relativ': relative_reden,
        'social_media_usage': social_media_usage_stacked,
        'Landtagswahlen': landtagswahlen_stacked
    }).dropna()

    # Zielvariable und X ausrichten
    y = y.reindex(X.index).dropna()
    X = X.loc[y.index]

    # Logistische Regression
    X = sm.add_constant(X)
    model = sm.Logit(y, X).fit()
    print(model.summary())

# Beispielaufruf
#logistic_regression_with_control(reden_relativ, post_relativ, post_common, social_media_usage)


In [17]:
# Mit fixed_effects
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, brier_score_loss, confusion_matrix
import statsmodels.api as sm
def log_reg_FE_control(relativ_rede, relativ_posts, post_to_shift,shifts, social_media_usage):
    """
    Führt eine logistische Regression mit Fixed Effects für Themen durch.
    
    Args:
    - relativ_rede: DataFrame mit Erwähnungen im Parlament; relative Anteile.
    - relativ_posts: DataFrame mit Erwähnungen auf Social Media; relative Anteile.
    """
    import statsmodels.api as sm

    # Zielvariable (t+1): Thema auf Social Media erwähnt, einen Tag nach Besprechung im Parlament
    posts_shifted_common = post_to_shift.shift(shifts).dropna()
    common_dates2 = relativ_rede.index.intersection(posts_shifted_common.index)
    relativ_rede = relativ_rede.loc[common_dates2]
    relativ_posts=relativ_posts.loc[common_dates2]
    social_media_usage = social_media_usage.loc[common_dates2]
    landtagswahlen = landtagswahlen_series.loc[common_dates2]
    y = (posts_shifted_common > 0).astype(int).stack()

    # Unabhängige Variablen: relative Anteile im Parlament und auf Social Media
    relative_reden = relativ_rede.stack()
    relative_posts = relativ_posts.stack()
    # Kontrollvariable Social Media hinzufügen
    social_media_usage_stacked = social_media_usage.reindex(relativ_rede.index)
    social_media_usage_stacked = social_media_usage_stacked.repeat(relativ_rede.shape[1])
    social_media_usage_stacked.index = relative_reden.index

    # Kontrollvariable Landtagswahlen hinzufügen
    # Dummy-Variable für Landtagswahlen
    landtagswahlen_stacked = landtagswahlen.reindex(relativ_rede.index)
    landtagswahlen_stacked = landtagswahlen_stacked.repeat(relativ_rede.shape[1])
    landtagswahlen_stacked.index = relative_reden.index
    # Zielvariable und unabhängige Variablen als DataFrame zusammenführen
    X = pd.DataFrame({'issue attention Facebook': relative_posts, 
                      'issue attention Bundestag': relative_reden,
                      'Social Media Nutzung': social_media_usage_stacked,
                       'Landtagswahlen': landtagswahlen_stacked }).dropna()
    y = y.reindex(X.index).dropna()

    # Konvertiere MultiIndex zu flachem Index, um die Kompatibilität sicherzustellen
    X.index = X.index.to_flat_index()
    y.index = y.index.to_flat_index()

    # Dummy-Variablen für Themen (Fixed Effects)
    topics = [idx[1] for idx in X.index]  # Der zweite Wert im Tupel repräsentiert das Topic
    topic_dummies = pd.get_dummies(topics, prefix='topic',drop_first=True).astype(int) # ohne.astype(int) kommen hier Fehlermeldungen
    # Versuch aus den Topics Topic 0 zu löschen und somit bessere Daten zu erlangen

    # Füge Dummy-Variablen hinzu
    X = pd.concat([X.reset_index(drop=True), topic_dummies.reset_index(drop=True)], axis=1)

    # Logistische Regression
    X = sm.add_constant(X)
    model = sm.Logit(y.values, X).fit(maxiter=100)
    print(model.summary())
    # Modellgüte-Kennzahlen berechnen
    y_pred_prob = model.predict(X)  # Vorhergesagte Wahrscheinlichkeiten
    y_pred = (y_pred_prob >= 0.5).astype(int)  # Binäre Vorhersagen

    # AUC-ROC
    auc_roc = roc_auc_score(y, y_pred_prob)
    print(f"AUC-ROC: {auc_roc}")

    # Präzision, Recall und F1-Score
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    print(f"Präzision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")

    # Brier-Score
    brier_score = brier_score_loss(y, y_pred_prob)
    print(f"Brier-Score: {brier_score}")

    # Confusion-Matrix
    cm = confusion_matrix(y, y_pred)
    print("Confusion-Matrix:")
    print(cm)

    return model, auc_roc, f1

In [18]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, brier_score_loss, confusion_matrix
import statsmodels.api as sm
import pandas as pd

def log_reg_FE_control_test(relativ_rede, relativ_posts, post_to_shift, shifts, social_media_usage, complexity_rede, complexity_post):
    """
    Führt eine logistische Regression mit Fixed Effects für Themen durch, inklusive Kontrolle für Komplexität.

    Args:
    - relativ_rede: DataFrame mit Erwähnungen im Parlament; relative Anteile.
    - relativ_posts: DataFrame mit Erwähnungen auf Social Media; relative Anteile.
    - post_to_shift: DataFrame der Social-Media-Posts zur Erstellung der Zielvariablen (verschoben um t+1).
    - shifts: Anzahl der Tage, um die Zielvariable zu verschieben.
    - social_media_usage: DataFrame mit der Anzahl der täglichen Social-Media-Beiträge.
    - landtagswahlen_series: Series mit Dummy-Variablen für Landtagswahlen.
    - complexity_rede: DataFrame mit der Komplexität (z. B. Wortanzahl) pro Tag für Reden.
    - complexity_post: DataFrame mit der Komplexität (z. B. Wortanzahl) pro Tag für Social-Media-Posts.

    Returns:
    - model: Das trainierte logistische Regressionsmodell.
    - auc_roc: Der AUC-ROC-Wert des Modells.
    - f1: Der F1-Score des Modells.
    """
    # Zielvariable (t+1): Thema auf Social Media erwähnt, einen Tag nach Besprechung im Parlament
    posts_shifted_common = post_to_shift.shift(shifts).dropna()
    common_dates2 = relativ_rede.index.intersection(posts_shifted_common.index)

    # Synchronisiere alle relevanten Daten mit den gemeinsamen Datenpunkten
    relativ_rede = relativ_rede.loc[common_dates2]
    relativ_posts = relativ_posts.loc[common_dates2]
    social_media_usage = social_media_usage.loc[common_dates2]
    landtagswahlen = landtagswahlen_series.loc[common_dates2]
    complexity_rede = complexity_rede.loc[common_dates2]
    complexity_post = complexity_post.loc[common_dates2]
    landtagswahlen = landtagswahlen_series.loc[common_dates2]

    # Zielvariable erstellen
    y = (posts_shifted_common > 0).astype(int).stack()

    # Unabhängige Variablen erstellen
    relative_reden = relativ_rede.stack()
    relative_posts = relativ_posts.stack()

    # Kontrollvariable: Social Media Nutzung
    social_media_usage_stacked = social_media_usage.reindex(relativ_rede.index)
    social_media_usage_stacked = social_media_usage_stacked.repeat(relativ_rede.shape[1])
    social_media_usage_stacked.index = relative_reden.index

    # Kontrollvariable: Landtagswahlen
    landtagswahlen_stacked = landtagswahlen.reindex(relativ_rede.index)
    landtagswahlen_stacked = landtagswahlen_stacked.repeat(relativ_rede.shape[1])
    landtagswahlen_stacked.index = relative_reden.index

    # Kontrollvariable: Komplexität der Reden (Z-Scores)
    complexity_rede_z = (complexity_rede - complexity_rede.mean()) / complexity_rede.std()
    complexity_rede_stacked = complexity_rede_z.reindex(relativ_rede.index)
    complexity_rede_stacked = complexity_rede_stacked.repeat(relativ_rede.shape[1])
    complexity_rede_stacked.index = relative_reden.index

    # Kontrollvariable: Komplexität der Posts (Z-Scores)
    complexity_post_z = (complexity_post - complexity_post.mean()) / complexity_post.std()
    complexity_post_stacked = complexity_post_z.reindex(relativ_posts.index)
    complexity_post_stacked = complexity_post_stacked.repeat(relativ_posts.shape[1])
    complexity_post_stacked.index = relative_posts.index

    # Zielvariable und unabhängige Variablen zusammenführen
    X = pd.DataFrame({
        'issue attention Facebook': relative_posts,
        'issue attention Bundestag': relative_reden,
        'Social Media Nutzung': social_media_usage_stacked,
        'Landtagswahlen': landtagswahlen_stacked,
        'Komplexität Reden': complexity_rede_stacked,
        'Komplexität Posts': complexity_post_stacked
    }).dropna()

    y = y.reindex(X.index).dropna()

    # Konvertiere MultiIndex zu flachem Index, um die Kompatibilität sicherzustellen
    X.index = X.index.to_flat_index()
    y.index = y.index.to_flat_index()

    # Dummy-Variablen für Themen (Fixed Effects)
    topics = [idx[1] for idx in X.index]  # Der zweite Wert im Tupel repräsentiert das Topic
    topic_dummies = pd.get_dummies(topics, prefix='topic', drop_first=True).astype(int)

    # Füge Dummy-Variablen hinzu
    X = pd.concat([X.reset_index(drop=True), topic_dummies.reset_index(drop=True)], axis=1)

    # Logistische Regression
    X = sm.add_constant(X)
    model = sm.Logit(y.values, X).fit(maxiter=100)

    print(model.summary())

    # Modellgüte-Kennzahlen berechnen
    y_pred_prob = model.predict(X)  # Vorhergesagte Wahrscheinlichkeiten
    y_pred = (y_pred_prob >= 0.5).astype(int)  # Binäre Vorhersagen

    # AUC-ROC
    auc_roc = roc_auc_score(y, y_pred_prob)
    print(f"AUC-ROC: {auc_roc}")

    # Präzision, Recall und F1-Score
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    print(f"Präzision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")

    # Brier-Score
    brier_score = brier_score_loss(y, y_pred_prob)
    print(f"Brier-Score: {brier_score}")

    # Confusion-Matrix
    cm = confusion_matrix(y, y_pred)
    print("Confusion-Matrix:")
    print(cm)

    return model, auc_roc, f1


In [19]:
log_reg_FE_control_test(rede_relativ_reduced, post_relativ_reduced, post_common,1, social_media_usage,rede_komplex, posts_komplex)

Optimization terminated successfully.
         Current function value: 0.531963
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2234
Method:                           MLE   Df Model:                           40
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2321
Time:                        11:03:15   Log-Likelihood:                -1210.2
converged:                       True   LL-Null:                       -1576.1
Covariance Type:            nonrobust   LLR p-value:                5.544e-128
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         2.9640      0.853      3.474      0.001       1.

(<statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x168b712b710>,
 0.802870277561223,
 0.6955266955266955)

In [20]:
# Schleife für logistische Regression mit Lags von -7 bis 7
for n in range(1, 8):  # Einschließlich 1 bis 7
    try:
        print(f"\nRegression für Lag {n}:")
        log_reg_FE_control(rede_relativ_reduced, post_relativ_reduced, post_common,n, social_media_usage)
    except Exception as e:
        print(f"Fehler bei Lag {n}: {e}")



Regression für Lag 1:
Optimization terminated successfully.
         Current function value: 0.532827
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2236
Method:                           MLE   Df Model:                           38
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2309
Time:                        11:03:15   Log-Likelihood:                -1212.2
converged:                       True   LL-Null:                       -1576.1
Covariance Type:            nonrobust   LLR p-value:                1.858e-128
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         2.6127      0.765      3.

##### Troubleshooting
Mit dem Code scheint etwas nicht funktionieren mit den durch die dummy-variablen erzeugten Werten. Daher hier nochmal ein trouble shooting. Das Problem lag darin, dass die hinzugefügten Dummy-Variablen nicht 0/1, sondern True/False codiert und somit für die Funktion nicht lesbar waren. Auch mit funktionierenden Dummy.Variablen ergeben die Ergebnisse nicht viel Sinn, da die ganzen Dummies zu Fehlern in der Berechnung führen. Ich versuche es nochmal mit dem Ansatz Themen, die zu mehr als 90% aus 0 bestehen aus dem Datensatz zu löschen und dann nochmal zu rechnen


In [21]:
logistic_regression_with_lags_SM_fixed_effects(rede_relativ_reduced, post_relativ_reduced,post_common)

Optimization terminated successfully.
         Current function value: 0.533088
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2238
Method:                           MLE   Df Model:                           36
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2305
Time:                        11:03:16   Log-Likelihood:                -1212.8
converged:                       True   LL-Null:                       -1576.1
Covariance Type:            nonrobust   LLR p-value:                1.616e-129
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              2.3898      0.734      3.258      0.001       0.952       3.828
posts_relative   

In [22]:
# Logistische Regression mit Interaktionseffekt
def logistic_regression_with_interaction(relativ_rede, relativ_posts, post_to_shift, social_media_usage):
    """
    Führt eine logistische Regression mit einem Interaktionseffekt durch.
    
    Args:
    - relativ_rede: DataFrame mit relativen Anteilen der Themen im Bundestag.
    - relativ_posts: DataFrame mit relativen Anteilen der Themen auf Social Media.
    - post_to_shift: DataFrame mit Themen auf Social Media (Zielvariable, t+1).
    - social_media_usage: Series mit täglicher Social-Media-Aktivität (Kontrollvariable).
    """
    import statsmodels.api as sm

    # Zielvariable (t+1): Thema auf Social Media erwähnt, einen Tag nach Besprechung im Parlament
    posts_shifted_common = post_to_shift.shift(1).dropna()
    common_dates2 = relativ_rede.index.intersection(posts_shifted_common.index)
    relativ_rede = relativ_rede.loc[common_dates2]
    relativ_posts = relativ_posts.loc[common_dates2]
    social_media_usage = social_media_usage.loc[common_dates2]

    y = (posts_shifted_common > 0).astype(int).stack()

    # Unabhängige Variablen
    relative_reden = relativ_rede.stack()
    relative_posts = relativ_posts.stack()

    # Kontrollvariable hinzufügen
    social_media_usage_stacked = social_media_usage.reindex(relativ_rede.index)
    social_media_usage_stacked = social_media_usage_stacked.repeat(relativ_rede.shape[1])
    social_media_usage_stacked.index = relative_reden.index

    # Interaktionseffekt berechnen
    interaction_effect = relative_reden * relative_posts

    # DataFrame für die logistische Regression
    X = pd.DataFrame({
        'posts_relative': relative_posts, 
        'reden_relativ': relative_reden,
        'social_media_usage': social_media_usage_stacked,
        'interaction': interaction_effect,
    }).dropna()

    # Zielvariable und X ausrichten
    y = y.reindex(X.index).dropna()
    X = X.loc[y.index]

    # Logistische Regression
    X = sm.add_constant(X)
    model = sm.Logit(y, X).fit()
    print(model.summary())

# Beispielaufruf
logistic_regression_with_interaction(reden_relativ, post_relativ, post_common, social_media_usage)


Optimization terminated successfully.
         Current function value: 0.598166
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2270
Method:                           MLE   Df Model:                            4
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.1366
Time:                        11:03:16   Log-Likelihood:                -1360.8
converged:                       True   LL-Null:                       -1576.1
Covariance Type:            nonrobust   LLR p-value:                 7.031e-92
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.6505      0.200     -3.260      0.001      -1.042      -0.259
posts

In [23]:
# Beispielaufruf
logistic_regression_with_control(reden_relativ, post_relativ, post_common, social_media_usage)


Optimization terminated successfully.
         Current function value: 0.598774
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2270
Method:                           MLE   Df Model:                            4
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.1357
Time:                        11:03:16   Log-Likelihood:                -1362.2
converged:                       True   LL-Null:                       -1576.1
Covariance Type:            nonrobust   LLR p-value:                 2.786e-91
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.6280      0.199     -3.162      0.002      -1.017      -0.239
posts

In [24]:
from stargazer.stargazer import Stargazer
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control(rede_relativ_reduced, post_relativ_reduced, post_common,lag, social_media_usage)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison.html", "w") as f:
    f.write(stargazer.render_html())


Optimization terminated successfully.
         Current function value: 0.532827
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2236
Method:                           MLE   Df Model:                           38
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2309
Time:                        11:03:16   Log-Likelihood:                -1212.2
converged:                       True   LL-Null:                       -1576.1
Covariance Type:            nonrobust   LLR p-value:                1.858e-128
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         2.6127      0.765      3.416      0.001       1.

##### Interpretation mit Interaktionseffekt (ChatGPT lol)
Vergleich der Modelle: Mit und ohne Interaktionseffekt

Hier ist die Interpretation der beiden Modelle und des Einflusses des Interaktionseffekts:
Modell ohne Interaktionseffekt
Ergebnisse:

    posts_relative (Koeffizient: 35.6484, p < 0.001):
        Die relative Aufmerksamkeit eines Themas auf Social Media (am gleichen Tag) hat einen starken und signifikanten positiven Einfluss darauf, ob das Thema am nächsten Tag auf Social Media wieder auftaucht.
        Einheitenanstieg von posts_relative (bei konstanten anderen Variablen) führt zu einer erheblichen Erhöhung der Wahrscheinlichkeit, dass das Thema erneut auf Social Media erscheint.

    reden_relativ (Koeffizient: 2.2387, p = 0.029):
        Die relative Aufmerksamkeit eines Themas im Bundestag am Vortag hat ebenfalls einen signifikanten positiven Einfluss.
        Dieser Effekt ist moderat im Vergleich zu posts_relative, bleibt aber relevant.

    social_media_usage (Koeffizient: -0.0014, p = 0.634):
        Die allgemeine Social-Media-Aktivität ist nicht signifikant. Dies deutet darauf hin, dass die Gesamtzahl der Posts keinen direkten Einfluss auf die Wahrscheinlichkeit hat, ob ein Thema auf Social Media erscheint.

Pseudo-R²:

    0.1357: Das Modell erklärt etwa 13.57% der Varianz in der Zielvariable.

Modell mit Interaktionseffekt
Ergebnisse:

    posts_relative (Koeffizient: 37.2377, p < 0.001):
        Die Bedeutung von posts_relative bleibt signifikant und ähnlich stark wie im Basismodell.

    reden_relativ (Koeffizient: 3.6387, p = 0.004):
        Der Effekt von reden_relativ bleibt positiv und signifikant, aber der Koeffizient ist größer als im Basismodell. Das deutet darauf hin, dass der Effekt von reden_relativ isoliert von seiner Interaktion mit posts_relative stärker wahrgenommen wird.

    social_media_usage (Koeffizient: -0.0014, p = 0.633):
        Wie im Basismodell kein signifikanter Einfluss.

    interaction (Koeffizient: -51.4108, p = 0.045):
        Der Interaktionseffekt ist signifikant und negativ.
        Das bedeutet, dass die gleichzeitige hohe Aufmerksamkeit eines Themas sowohl im Bundestag als auch auf Social Media (am gleichen Tag) die Wahrscheinlichkeit senkt, dass dieses Thema am nächsten Tag erneut auf Social Media erscheint.

Pseudo-R²:

    0.1366: Das Modell erklärt etwa 13.66% der Varianz in der Zielvariable, ein geringfügiger Anstieg im Vergleich zum Basismodell.

Interpretation des Interaktionseffekts

Der Interaktionseffekt legt nahe:

    Wenn ein Thema gleichzeitig hohe Aufmerksamkeit in Bundestagsreden und auf Social Media erhält, sinkt die Wahrscheinlichkeit, dass das Thema am nächsten Tag erneut auf Social Media erscheint.
    Dies könnte darauf hindeuten, dass Themen, die bereits vollständig behandelt und ausgeschöpft wirken (auf beiden Plattformen), weniger relevant erscheinen und dadurch weniger erneut gepostet werden.

Vergleich der Modellgüte

    Pseudo-R²:
        Modell mit Interaktion: 0.1366
        Basismodell: 0.1357
        Der Anstieg ist minimal, was bedeutet, dass der Interaktionseffekt nur wenig zusätzliche Erklärungskraft bietet.

    Log-Likelihood:
        Modell mit Interaktion: -1360.8
        Basismodell: -1362.2
        Ein besserer Wert im Modell mit Interaktion, was ebenfalls auf eine geringfügige Verbesserung hindeutet.

Fazit

    Der Interaktionseffekt ist statistisch signifikant, seine zusätzliche Erklärungskraft ist jedoch marginal.
    Inhaltlich ist der Effekt interessant, da er zeigt, dass eine gleichzeitige hohe Aufmerksamkeit eines Themas auf beiden Plattformen zu einer "Sättigung" führen könnte.
    Ob der Interaktionseffekt behalten werden sollte, hängt davon ab, wie wichtig diese Interpretation für deine Fragestellung ist.

##### Pro/ Contra Interaktionseffekt
Die Integration des Interaktionseffekts verändert die Interpretation des Modells auf subtile, aber inhaltlich wichtige Weise. Hier ist eine Analyse, wie sich die Interpretation verändert, und was für oder gegen das Behalten des Interaktionseffekts spricht:

---

### **Veränderte Interpretation im Vergleich zum Basismodell**

#### **Basismodell:**
1. **`posts_relative` und `reden_relativ`:**
   - Beide Variablen haben einen positiven und signifikanten Einfluss.
   - Sie wirken additiv: Ein höherer Wert von `posts_relative` oder `reden_relativ` führt unabhängig voneinander zu einer erhöhten Wahrscheinlichkeit, dass ein Thema am nächsten Tag auf Social Media auftaucht.
   - Es wird angenommen, dass die Effekte voneinander unabhängig sind, d.h., sie beeinflussen sich nicht gegenseitig.

#### **Modell mit Interaktionseffekt:**
1. **`posts_relative` und `reden_relativ`:**
   - Die Haupteffekte bleiben bestehen, jedoch ist die Interpretation nuancierter:
     - **`reden_relativ`:** Der Effekt hängt jetzt davon ab, wie hoch `posts_relative` ist (und umgekehrt). 
     - Ein höherer Wert von `reden_relativ` hat einen stärkeren positiven Effekt, wenn `posts_relative` niedrig ist, verliert jedoch an Wirkung (und wird sogar negativ), wenn `posts_relative` hoch ist.

2. **Interaktionseffekt (`interaction`):**
   - Die negative Signifikanz des Interaktionseffekts bedeutet, dass die gleichzeitige hohe Aufmerksamkeit eines Themas in Bundestagsreden und auf Social Media die Wahrscheinlichkeit verringert, dass das Thema am nächsten Tag erneut auf Social Media auftaucht.
   - Dies kann darauf hindeuten, dass Themen, die bereits umfassend behandelt wurden (sowohl im Parlament als auch auf Social Media), für den nächsten Tag an Relevanz verlieren.

---

### **Argumente für das Behalten des Interaktionseffekts**
1. **Inhaltliche Plausibilität:**
   - Die Interaktion zwischen `reden_relativ` und `posts_relative` macht inhaltlich Sinn, wenn man annimmt, dass Themen, die in beiden Arenen gleichzeitig sehr präsent sind, schneller an Aufmerksamkeit verlieren.
   - Dies fügt eine zusätzliche Dimension hinzu und reflektiert die komplexe Dynamik zwischen Parlament und Social Media.

2. **Signifikanz:**
   - Der Interaktionseffekt ist statistisch signifikant (p = 0.045), was darauf hindeutet, dass er einen echten Einfluss hat.

3. **Zusätzliche Erklärungskraft:**
   - Obwohl der Anstieg im Pseudo-R² gering ist (von 0.1357 auf 0.1366), zeigt der Log-Likelihood-Vergleich eine Verbesserung des Modells mit Interaktion.
   - Das Modell mit Interaktion ist also statistisch etwas besser angepasst.

4. **Neue Perspektive:**
   - Die Berücksichtigung des Interaktionseffekts ermöglicht es, die Daten aus einer anderen Perspektive zu betrachten. Das Modell kann spezifische Hypothesen testen, wie `posts_relative` und `reden_relativ` gemeinsam wirken.

---

### **Argumente gegen das Behalten des Interaktionseffekts**
1. **Geringfügige Verbesserung:**
   - Die zusätzliche Erklärungskraft des Interaktionseffekts ist minimal. Der Pseudo-R²-Wert steigt nur marginal, was darauf hindeutet, dass der Effekt für die Gesamtmodellgüte nicht entscheidend ist.

2. **Komplexität:**
   - Das Modell wird komplexer, und die Interpretation wird schwieriger. Der negative Interaktionseffekt kann für Leser oder Anwender, die mit der Datenanalyse weniger vertraut sind, verwirrend sein.
   - Es könnte schwerer sein, die Ergebnisse praktisch oder politisch zu interpretieren.

3. **Robustheit:**
   - Die Signifikanz des Interaktionseffekts könnte in anderen Stichproben oder bei einer größeren Anzahl von Variablen verschwinden. Dies könnte darauf hindeuten, dass der Effekt nur in diesem spezifischen Datensatz relevant ist.

---

### **Fazit**
- **Für das Behalten:**
   - Wenn das Zusammenspiel von `reden_relativ` und `posts_relative` ein zentraler Bestandteil deiner Forschungsfrage ist, sollte der Interaktionseffekt beibehalten werden, da er wertvolle Erkenntnisse liefert.
   - Das Modell mit Interaktion ist inhaltlich reicher und ermöglicht es, differenzierte Hypothesen zu testen.

- **Für das Ignorieren:**
   - Wenn die zusätzliche Komplexität die praktische Anwendbarkeit oder Verständlichkeit des Modells erschwert, könnte das Basismodell ausreichen. Insbesondere, wenn der Fokus auf der individuellen Wirkung von `reden_relativ` und `posts_relative` liegt.

Empfehlung: Falls der Interaktionseffekt inhaltlich Sinn ergibt und du ihn testen möchtest, behalte ihn bei, aber stelle sicher, dass die Ergebnisse robust sind (z. B. durch Sensitivitätsanalysen).

##### Einbezug Social Media Nutzung als Kontrollvariable
Im Folgenden der Versuch, Social Media Nutzung miteinzubeziehen. Dabei wird diese zuerst generell miteinbezogen und dann nochmal auf die einzelnen Parteien aufgeschlüsselt

#### Regression aufgesplittet nach Parteien 2
Leider funktioniert mein Vorgehen nicht, da so scheinbar zu viele erklärende Variablen herangezogen werden und das Modell nicht richtig rechnen kann. Deswegen folgend der Split in eine Regression mit Partei postings und eine mit Parteireden und dann Überprüfung, welches der Modelle besser passt. Führt zu den gleichen Ergebnissen wie zuvor, weshalb anschließend nochmal geschaut wird, wie man die Daten bereinigen kann, um Störfaktoren zu entfernen.
Update: Auch diese Vorgehen funktioniert leider nicht, weshalb ich das jetzt erstmal nicht mehr weiterverfolge!

#### Aufteilen der Daten nach einzelnen Parteien
Hier bilde ich jetzt Subdatensätze für die einzelnen Parteien und führe für diese einzelne Regressionen durch. Prinzipiell muss ich nochmal nach Kontrollvariablen schauen

In [25]:
# Beispiel: Die Funktion aufrufen
#post_party_relativ = post_party_relativ.drop(columns=['relative_post_Bundesregierung', 'relative_post_CSU'])
# log_reg_single_party(post_party_relativ)

#### AfD

In [26]:
subset_posts_afd = subset_posts[subset_posts["partei"] == "AfD"]
subset_reden_afd = subset_reden[subset_reden["partei"] == "AfD"]
social_media_usage_afd = subset_posts_afd.groupby('date').size()
subset_reden_afd["date"] = pd.to_datetime(subset_reden_afd["date"])
subset_posts_afd["date"] = pd.to_datetime(subset_posts_afd["date"])
# 🔹 2️⃣ Gruppiere die Themen täglich und zähle die Häufigkeit der Themen pro Tag
redethemen_täglich_afd = subset_reden_afd.groupby(['date', 'Topic']).size().unstack(fill_value=0)
postthemen_täglich_afd = subset_posts_afd.groupby(['date', 'Topic']).size().unstack(fill_value=0)
reden_komplexität_täglich_afd = subset_reden_afd.groupby('date')['komplexität'].sum()
posts_komplexität_täglich_afd = subset_posts_afd.groupby('date')['komplexität'].sum()
# Führe einen inner join durch, damit nur gemeinsame Tage beibehalten werden
common_dates_afd = redethemen_täglich_afd.index.intersection(postthemen_täglich_afd.index)
redethemen_täglich_aligned_afd = redethemen_täglich_afd.loc[common_dates_afd]
postthemen_täglich_aligned_afd = postthemen_täglich_afd.loc[common_dates_afd]
rede_komplex_afd = reden_komplexität_täglich_afd.loc[common_dates_afd]
posts_komplex_afd = posts_komplexität_täglich_afd.loc[common_dates_afd]
# Zielvariable: Facebookposts um einen Tag in die Zukunft gelaggt
# # posts_shifted enthält die Facebookposts mit lag t+1, Die Daten hier sind zwar die gleichen wie in den anderen Dataframes,
# # die Werte jedoch die vom Vortag, weshalb die Analyse zulässig ist
# posts_shifted_afd=postthemen_täglich_aligned_afd.shift().dropna()
# # Erneute Anpassung des Datums
# common_dates2_afd = redethemen_täglich_afd.index.intersection(posts_shifted_afd.index)
# redethemen_täglich_aligned_afd = redethemen_täglich_afd.loc[common_dates2]
# postthemen_täglich_aligned_afd = postthemen_täglich_afd.loc[common_dates2]
# rede_common_afd, post_common_afd = filter_common_topics(redethemen_täglich_aligned_afd, postthemen_täglich_aligned_afd)
# posts_shifted_common_afd, post_common_afd = filter_common_topics(post_common_afd, posts_shifted_afd)
# # Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
# reden_relativ_afd =  rede_common_afd.div(rede_common_afd.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
# post_relativ_afd = post_common_afd.div(post_common_afd.sum(axis=1), axis=0)

C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\1352934036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_reden_afd["date"] = pd.to_datetime(subset_reden_afd["date"])
C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\1352934036.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_posts_afd["date"] = pd.to_datetime(subset_posts_afd["date"])


In [27]:
rede_common_afd, post_common_afd = filter_common_topics(redethemen_täglich_aligned_afd, postthemen_täglich_aligned_afd)
# Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
reden_relativ_afd =  rede_common_afd.div(rede_common_afd.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
post_relativ_afd = post_common_afd.div(post_common_afd.sum(axis=1), axis=0)
reden_relativ_afd_red = remove_near_constant(reden_relativ_afd)
post_relativ_afd_red = remove_near_constant(post_relativ_afd)
rede_reduced_afd, post_reduced_afd = filter_common_topics(reden_relativ_afd_red, post_relativ_afd_red)

Gemeinsame Topics: {0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 34, 35, 36, 37, 40, 43, 44, 46, 51, 52, 53, 54, 56, 58, 59, 61, 63, 64, 65, 68, 69, 70, 72, 74, 75, 77, 78, 84, 88, 90, 91, 92}
Gemeinsame Topics: {0, 1, 2, 3, 4, 6, 7, 8, 10, 14, 15, 29}


In [28]:
logistic_regression_with_lags_SM(reden_relativ_afd,post_relativ_afd, post_common_afd)

Optimization terminated successfully.
         Current function value: 0.374519
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 3965
Model:                          Logit   Df Residuals:                     3962
Method:                           MLE   Df Model:                            2
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.1704
Time:                        11:03:16   Log-Likelihood:                -1485.0
converged:                       True   LL-Null:                       -1790.0
Covariance Type:            nonrobust   LLR p-value:                3.502e-133
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.1491      0.055    -39.036      0.000      -2.257      -2.041
posts_relative   

In [29]:
logistic_regression_with_lags_SM_fixed_effects(rede_reduced_afd,post_relativ_afd, post_common_afd)

Optimization terminated successfully.
         Current function value: 0.488912
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  780
Model:                          Logit   Df Residuals:                      766
Method:                           MLE   Df Model:                           13
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2920
Time:                        11:03:17   Log-Likelihood:                -381.35
converged:                       True   LL-Null:                       -538.64
Covariance Type:            nonrobust   LLR p-value:                 2.125e-59
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              1.3713      0.394      3.478      0.001       0.598       2.144
posts_relative   

In [30]:
log_reg_FE_control(rede_reduced_afd,post_reduced_afd, post_common_afd,1,social_media_usage_afd)

Optimization terminated successfully.
         Current function value: 0.487541
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  780
Model:                          Logit   Df Residuals:                      764
Method:                           MLE   Df Model:                           15
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2940
Time:                        11:03:17   Log-Likelihood:                -380.28
converged:                       True   LL-Null:                       -538.64
Covariance Type:            nonrobust   LLR p-value:                 1.855e-58
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         1.5478      0.562      2.754      0.006       0.

(<statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x168b28ffc80>,
 0.8382722250125565,
 0.7341389728096677)

In [31]:
# Schleife für logistische Regression mit Lags von -7 bis 7
for n in range(1, 8):  # Einschließlich 1 bis 7
    try:
        print(f"\nRegression für Lag {n}:")
        log_reg_FE_control(rede_reduced_afd, post_reduced_afd, post_common_afd,n, social_media_usage_afd)
    except Exception as e:
        print(f"Fehler bei Lag {n}: {e}")



Regression für Lag 1:
Optimization terminated successfully.
         Current function value: 0.487541
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  780
Model:                          Logit   Df Residuals:                      764
Method:                           MLE   Df Model:                           15
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2940
Time:                        11:03:17   Log-Likelihood:                -380.28
converged:                       True   LL-Null:                       -538.64
Covariance Type:            nonrobust   LLR p-value:                 1.855e-58
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         1.5478      0.562      2.

In [32]:
## Anteil AfD an gesamten Postingverhalten
# Wie sinnvoll? Der meiste Output kam eh von der AfD, dass hier dann hohe Werte rauskommen ist klar
log_reg_FE_control(rede_reduced_afd,post_reduced_afd, post_common_afd,1,social_media_usage_afd)

Optimization terminated successfully.
         Current function value: 0.487541
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  780
Model:                          Logit   Df Residuals:                      764
Method:                           MLE   Df Model:                           15
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2940
Time:                        11:03:17   Log-Likelihood:                -380.28
converged:                       True   LL-Null:                       -538.64
Covariance Type:            nonrobust   LLR p-value:                 1.855e-58
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         1.5478      0.562      2.754      0.006       0.

(<statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x168b725a6c0>,
 0.8382722250125565,
 0.7341389728096677)

In [33]:
models_afd = []
metrics_afd = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model_afd, auc_roc_afd, f1_afd = log_reg_FE_control(rede_reduced_afd,post_reduced_afd, post_common_afd,lag,social_media_usage_afd)
    models_afd.append(model_afd)
    metrics_afd.append((auc_roc_afd, f1_afd))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models_afd)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - AfD")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes_afd = [f"Lag {i}: AUC-ROC = {metrics_afd[i-1][0]:.3f}, F1-Score = {metrics_afd[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes_afd)


# Exportieren
with open("regression_table_comparison_afd.html", "w") as f:
    f.write(stargazer.render_html())


Optimization terminated successfully.
         Current function value: 0.487541
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  780
Model:                          Logit   Df Residuals:                      764
Method:                           MLE   Df Model:                           15
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2940
Time:                        11:03:17   Log-Likelihood:                -380.28
converged:                       True   LL-Null:                       -538.64
Covariance Type:            nonrobust   LLR p-value:                 1.855e-58
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         1.5478      0.562      2.754      0.006       0.

In [34]:
models_afd_complex = []
metrics_afd_complex = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model_afd, auc_roc_afd, f1_afd = log_reg_FE_control_test(rede_reduced_afd,post_reduced_afd, post_common_afd,lag,social_media_usage_afd, rede_komplex_afd, posts_komplex_afd)
    models_afd_complex.append(model_afd)
    metrics_afd_complex.append((auc_roc_afd, f1_afd))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models_afd_complex)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - AfD")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes_afd = [f"Lag {i}: AUC-ROC = {metrics_afd_complex[i-1][0]:.3f}, F1-Score = {metrics_afd_complex[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes_afd)


# Exportieren
with open("regression_table_comparison_afd_complex.html", "w") as f:
    f.write(stargazer.render_html())


Optimization terminated successfully.
         Current function value: 0.486842
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  780
Model:                          Logit   Df Residuals:                      762
Method:                           MLE   Df Model:                           17
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.2950
Time:                        11:03:18   Log-Likelihood:                -379.74
converged:                       True   LL-Null:                       -538.64
Covariance Type:            nonrobust   LLR p-value:                 2.346e-57
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         1.3009      0.781      1.665      0.096      -0.

#### SPD


In [35]:
subset_posts_spd = subset_posts[subset_posts["partei"] == "SPD"]
subset_reden_spd = subset_reden[subset_reden["partei"] == "SPD"]
social_media_usage_spd = subset_posts_spd.groupby('date').size()
subset_reden_spd["date"] = pd.to_datetime(subset_reden_spd["date"])
subset_posts_afd["date"] = pd.to_datetime(subset_posts_spd["date"])
reden_komplexität_täglich_spd = subset_reden_spd.groupby('date')['komplexität'].sum()
posts_komplexität_täglich_spd = subset_posts_spd.groupby('date')['komplexität'].sum()
# 🔹 2️⃣ Gruppiere die Themen täglich und zähle die Häufigkeit der Themen pro Tag
redethemen_täglich_spd = subset_reden_spd.groupby(['date', 'Topic']).size().unstack(fill_value=0)
postthemen_täglich_spd = subset_posts_spd.groupby(['date', 'Topic']).size().unstack(fill_value=0)
# Führe einen inner join durch, damit nur gemeinsame Tage beibehalten werden
common_dates_spd = redethemen_täglich_spd.index.intersection(postthemen_täglich_spd.index)
redethemen_täglich_aligned_spd = redethemen_täglich_spd.loc[common_dates_spd]
postthemen_täglich_aligned_spd = postthemen_täglich_spd.loc[common_dates_spd]
rede_komplex_spd = reden_komplexität_täglich_spd.loc[common_dates_spd]
posts_komplex_spd = posts_komplexität_täglich_spd.loc[common_dates_spd]

C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\577048389.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_reden_spd["date"] = pd.to_datetime(subset_reden_spd["date"])
C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\577048389.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_posts_afd["date"] = pd.to_datetime(subset_posts_spd["date"])


In [36]:
rede_common_spd, post_common_spd = filter_common_topics(redethemen_täglich_aligned_spd, postthemen_täglich_aligned_spd)
# Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
reden_relativ_spd =  rede_common_spd.div(rede_common_spd.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
post_relativ_spd = post_common_spd.div(post_common_spd.sum(axis=1), axis=0)
reden_relativ_spd_red = remove_near_constant(reden_relativ_spd)
post_relativ_spd_red = remove_near_constant(post_relativ_spd)
rede_reduced_spd, post_reduced_spd = filter_common_topics(reden_relativ_spd_red, post_relativ_spd_red)

Gemeinsame Topics: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39, 40, 41, 43, 44, 46, 47, 54, 55, 56, 58, 59, 61, 63, 64, 65, 67, 68, 69, 70, 75, 77, 78, 79, 80, 83, 84, 90, 91, 92}
Gemeinsame Topics: {0, 1, 2, 4, 6, 7, 8, 15, 19, 24}


In [37]:
logistic_regression_with_lags_SM(rede_reduced_spd,post_reduced_spd, post_common_spd)

Optimization terminated successfully.
         Current function value: 0.541385
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  640
Model:                          Logit   Df Residuals:                      637
Method:                           MLE   Df Model:                            2
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                0.009090
Time:                        11:03:18   Log-Likelihood:                -346.49
converged:                       True   LL-Null:                       -349.67
Covariance Type:            nonrobust   LLR p-value:                   0.04165
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.3009      0.113    -11.540      0.000      -1.522      -1.080
posts_relative   

In [38]:
logistic_regression_with_lags_SM_fixed_effects(rede_reduced_spd,post_reduced_spd, post_common_spd)

Optimization terminated successfully.
         Current function value: 0.504419
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  640
Model:                          Logit   Df Residuals:                      628
Method:                           MLE   Df Model:                           11
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                 0.07675
Time:                        11:03:18   Log-Likelihood:                -322.83
converged:                       True   LL-Null:                       -349.67
Covariance Type:            nonrobust   LLR p-value:                 1.353e-07
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.7140      0.283     -2.523      0.012      -1.269      -0.159
posts_relative   

In [39]:
# Schleife für logistische Regression mit Lags von -7 bis 7
for n in range(1, 8):  # Einschließlich 1 bis 7
    try:
        print(f"\nRegression für Lag {n}:")
        log_reg_FE_control(rede_reduced_spd, post_reduced_spd, post_common_spd,n, social_media_usage_spd)
    except Exception as e:
        print(f"Fehler bei Lag {n}: {e}")


Regression für Lag 1:
Optimization terminated successfully.
         Current function value: 0.503442
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  640
Model:                          Logit   Df Residuals:                      626
Method:                           MLE   Df Model:                           13
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                 0.07854
Time:                        11:03:18   Log-Likelihood:                -322.20
converged:                       True   LL-Null:                       -349.67
Covariance Type:            nonrobust   LLR p-value:                 4.167e-07
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.9336      0.350     -2.

In [40]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control(rede_reduced_spd, post_reduced_spd, post_common_spd,lag, social_media_usage_spd)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - SPD")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_spd.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.503442
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  640
Model:                          Logit   Df Residuals:                      626
Method:                           MLE   Df Model:                           13
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                 0.07854
Time:                        11:03:19   Log-Likelihood:                -322.20
converged:                       True   LL-Null:                       -349.67
Covariance Type:            nonrobust   LLR p-value:                 4.167e-07
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.9336      0.350     -2.669      0.008      -1.

In [41]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control_test(rede_reduced_spd, post_reduced_spd, post_common_spd,lag, social_media_usage_spd,rede_komplex_spd,posts_komplex_spd)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - SPD")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_spd_complex.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.500824
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  640
Model:                          Logit   Df Residuals:                      624
Method:                           MLE   Df Model:                           15
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                 0.08333
Time:                        11:03:19   Log-Likelihood:                -320.53
converged:                       True   LL-Null:                       -349.67
Covariance Type:            nonrobust   LLR p-value:                 4.978e-07
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -1.6471      0.531     -3.103      0.002      -2.

In [42]:
log_reg_FE_control_test()

TypeError: log_reg_FE_control_test() missing 7 required positional arguments: 'relativ_rede', 'relativ_posts', 'post_to_shift', 'shifts', 'social_media_usage', 'complexity_rede', and 'complexity_post'

#### CDU

In [43]:
subset_posts_cdu = subset_posts[subset_posts["partei"] == "CDU"]
subset_reden_cdu = subset_reden[subset_reden["partei"] == "CDU"]
social_media_usage_cdu = subset_posts_cdu.groupby('date').size()
subset_reden_cdu["date"] = pd.to_datetime(subset_reden_cdu["date"])
subset_posts_cdu["date"] = pd.to_datetime(subset_posts_cdu["date"])
reden_komplexität_täglich_cdu = subset_reden_cdu.groupby('date')['komplexität'].sum()
posts_komplexität_täglich_cdu = subset_posts_cdu.groupby('date')['komplexität'].sum()
# 🔹 2️⃣ Gruppiere die Themen täglich und zähle die Häufigkeit der Themen pro Tag
redethemen_täglich_cdu = subset_reden_cdu.groupby(['date', 'Topic']).size().unstack(fill_value=0)
postthemen_täglich_cdu = subset_posts_cdu.groupby(['date', 'Topic']).size().unstack(fill_value=0)
# Führe einen inner join durch, damit nur gemeinsame Tage beibehalten werden
common_dates_cdu = redethemen_täglich_cdu.index.intersection(postthemen_täglich_cdu.index)
redethemen_täglich_aligned_cdu = redethemen_täglich_cdu.loc[common_dates_cdu]
postthemen_täglich_aligned_cdu = postthemen_täglich_cdu.loc[common_dates_cdu]
rede_komplex_cdu = reden_komplexität_täglich_cdu.loc[common_dates_cdu]
posts_komplex_cdu = posts_komplexität_täglich_cdu.loc[common_dates_cdu]

C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\3344225774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_reden_cdu["date"] = pd.to_datetime(subset_reden_cdu["date"])
C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\3344225774.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_posts_cdu["date"] = pd.to_datetime(subset_posts_cdu["date"])


In [44]:
rede_common_cdu, post_common_cdu = filter_common_topics(redethemen_täglich_aligned_cdu, postthemen_täglich_aligned_cdu)
# Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
reden_relativ_cdu =  rede_common_cdu.div(rede_common_cdu.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
post_relativ_cdu = post_common_cdu.div(post_common_cdu.sum(axis=1), axis=0)
reden_relativ_cdu_red = remove_near_constant(reden_relativ_cdu)
post_relativ_cdu_red = remove_near_constant(post_relativ_cdu)
rede_reduced_cdu, post_reduced_cdu = filter_common_topics(reden_relativ_cdu_red, post_relativ_cdu_red)

Gemeinsame Topics: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 52, 54, 56, 58, 68, 69, 70, 74, 75, 77, 78, 80, 83, 84, 90, 92}
Gemeinsame Topics: {0, 1, 2, 3, 4, 6, 8, 10, 12, 14, 15, 20, 22}


In [561]:
logistic_regression_with_lags_SM(rede_reduced_cdu,post_reduced_cdu, post_common_cdu)

Optimization terminated successfully.
         Current function value: 0.585608
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  845
Model:                          Logit   Df Residuals:                      842
Method:                           MLE   Df Model:                            2
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.02739
Time:                        18:29:04   Log-Likelihood:                -494.84
converged:                       True   LL-Null:                       -508.77
Covariance Type:            nonrobust   LLR p-value:                 8.876e-07
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.1468      0.095    -12.118      0.000      -1.332      -0.961
posts_relative   

In [562]:
logistic_regression_with_lags_SM_fixed_effects(rede_reduced_cdu,post_reduced_cdu, post_common_cdu)

Optimization terminated successfully.


         Current function value: 0.566685
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  845
Model:                          Logit   Df Residuals:                      830
Method:                           MLE   Df Model:                           14
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.05882
Time:                        18:29:04   Log-Likelihood:                -478.85
converged:                       True   LL-Null:                       -508.77
Covariance Type:            nonrobust   LLR p-value:                 1.247e-07
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.5218      0.265     -1.967      0.049      -1.042      -0.002
posts_relative     3.5677      0.911      3.914      0

In [563]:
# Schleife für logistische Regression mit Lags von -7 bis 7
for n in range(1, 8):  # Einschließlich 1 bis 7
    try:
        print(f"\nRegression für Lag {n}:")
        log_reg_FE_control(rede_reduced_cdu, post_reduced_cdu, post_common_cdu,n, social_media_usage_cdu)
    except Exception as e:
        print(f"Fehler bei Lag {n}: {e}")


Regression für Lag 1:
Optimization terminated successfully.
         Current function value: 0.563633
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  845
Model:                          Logit   Df Residuals:                      828
Method:                           MLE   Df Model:                           16
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.06389
Time:                        18:29:04   Log-Likelihood:                -476.27
converged:                       True   LL-Null:                       -508.77
Covariance Type:            nonrobust   LLR p-value:                 7.346e-08
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.5468      0.326     -1.

In [564]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control(rede_reduced_cdu, post_reduced_cdu, post_common_cdu,lag, social_media_usage_cdu)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - CDU")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_cdu.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.563633
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  845
Model:                          Logit   Df Residuals:                      828
Method:                           MLE   Df Model:                           16
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.06389
Time:                        18:29:04   Log-Likelihood:                -476.27
converged:                       True   LL-Null:                       -508.77
Covariance Type:            nonrobust   LLR p-value:                 7.346e-08
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.5468      0.326     -1.677      0.094      -1.

In [565]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control_test(rede_reduced_cdu, post_reduced_cdu, post_common_cdu,lag, social_media_usage_cdu,rede_komplex_cdu,posts_komplex_cdu)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - CDU")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_cdu_complex.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.563529
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  845
Model:                          Logit   Df Residuals:                      826
Method:                           MLE   Df Model:                           18
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.06406
Time:                        18:29:05   Log-Likelihood:                -476.18
converged:                       True   LL-Null:                       -508.77
Covariance Type:            nonrobust   LLR p-value:                 2.899e-07
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.7274      0.574     -1.267      0.205      -1.

#### Bündnis 90/ DIE GRÜNEN

In [45]:
subset_posts_gruen = subset_posts[subset_posts["partei"] == "BÜNDNIS 90/DIE GRÜNEN"]
subset_reden_gruen = subset_reden[subset_reden["partei"] == "BÜNDNIS 90/DIE GRÜNEN"]
social_media_usage_gruen = subset_posts_gruen.groupby('date').size()
subset_reden_gruen["date"] = pd.to_datetime(subset_reden_gruen["date"])
subset_posts_gruen["date"] = pd.to_datetime(subset_posts_gruen["date"])
reden_komplexität_täglich_gruen = subset_reden_gruen.groupby('date')['komplexität'].sum()
posts_komplexität_täglich_gruen = subset_posts_gruen.groupby('date')['komplexität'].sum()
# 🔹 2️⃣ Gruppiere die Themen täglich und zähle die Häufigkeit der Themen pro Tag
redethemen_täglich_gruen = subset_reden_gruen.groupby(['date', 'Topic']).size().unstack(fill_value=0)
postthemen_täglich_gruen = subset_posts_gruen.groupby(['date', 'Topic']).size().unstack(fill_value=0)
# Führe einen inner join durch, damit nur gemeinsame Tage beibehalten werden
common_dates_gruen = redethemen_täglich_gruen.index.intersection(postthemen_täglich_gruen.index)
redethemen_täglich_aligned_gruen = redethemen_täglich_gruen.loc[common_dates_gruen]
postthemen_täglich_aligned_gruen = postthemen_täglich_gruen.loc[common_dates_gruen]
rede_komplex_gruen = reden_komplexität_täglich_gruen.loc[common_dates_gruen]
posts_komplex_gruen = posts_komplexität_täglich_gruen.loc[common_dates_gruen]

C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\982104320.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_reden_gruen["date"] = pd.to_datetime(subset_reden_gruen["date"])
C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\982104320.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_posts_gruen["date"] = pd.to_datetime(subset_posts_gruen["date"])


In [46]:
rede_common_gruen, post_common_gruen = filter_common_topics(redethemen_täglich_aligned_gruen, postthemen_täglich_aligned_gruen)
# Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
reden_relativ_gruen =  rede_common_gruen.div(rede_common_gruen.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
post_relativ_gruen = post_common_gruen.div(post_common_gruen.sum(axis=1), axis=0)
reden_relativ_gruen_red = remove_near_constant(reden_relativ_gruen)
post_relativ_gruen_red = remove_near_constant(post_relativ_gruen)
rede_reduced_gruen, post_reduced_gruen = filter_common_topics(reden_relativ_gruen_red, post_relativ_gruen_red)

Gemeinsame Topics: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 36, 37, 38, 41, 44, 46, 53, 54, 56, 58, 59, 61, 63, 65, 69, 70, 75, 79, 80, 81, 84, 90, 91, 92}
Gemeinsame Topics: {1, 2, 6, 15, 24, 25}


In [568]:
logistic_regression_with_lags_SM(rede_reduced_gruen,post_reduced_gruen, post_common_gruen)

Optimization terminated successfully.
         Current function value: 0.449798
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  384
Model:                          Logit   Df Residuals:                      381
Method:                           MLE   Df Model:                            2
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                0.001693
Time:                        18:29:05   Log-Likelihood:                -172.72
converged:                       True   LL-Null:                       -173.02
Covariance Type:            nonrobust   LLR p-value:                    0.7461
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.5605      0.156    -10.024      0.000      -1.866      -1.255
posts_relative   

In [569]:
logistic_regression_with_lags_SM_fixed_effects(rede_reduced_gruen,post_reduced_gruen, post_common_gruen)

Optimization terminated successfully.
         Current function value: 0.446520
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  384
Model:                          Logit   Df Residuals:                      376
Method:                           MLE   Df Model:                            7
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                0.008969
Time:                        18:29:05   Log-Likelihood:                -171.46
converged:                       True   LL-Null:                       -173.02
Covariance Type:            nonrobust   LLR p-value:                    0.8753
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.3112      0.324     -4.048      0.000      -1.946      -0.676
posts_relative   

In [570]:
# Schleife für logistische Regression mit Lags von -7 bis 7
for n in range(1, 8):  # Einschließlich 1 bis 7
    try:
        print(f"\nRegression für Lag {n}:")
        log_reg_FE_control(rede_reduced_gruen, post_reduced_gruen, post_common_gruen,n, social_media_usage_gruen)
    except Exception as e:
        print(f"Fehler bei Lag {n}: {e}")

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Regression für Lag 1:
Optimization terminated successfully.
         Current function value: 0.439438
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  384
Model:                          Logit   Df Residuals:                      374
Method:                           MLE   Df Model:                            9
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.02469
Time:                        18:29:05   Log-Likelihood:                -168.74
converged:                       True   LL-Null:                       -173.02
Covariance Type:            nonrobust   LLR p-value:                    0.4805
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -1.8715      0.423     -4.

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anac

Confusion-Matrix:
[[297   1]
 [ 62   0]]

Regression für Lag 6:
Optimization terminated successfully.
         Current function value: 0.455196
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  354
Model:                          Logit   Df Residuals:                      344
Method:                           MLE   Df Model:                            9
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                0.009454
Time:                        18:29:05   Log-Likelihood:                -161.14
converged:                       True   LL-Null:                       -162.68
Covariance Type:            nonrobust   LLR p-value:                    0.9612
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const         

In [571]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control(rede_reduced_gruen, post_reduced_gruen, post_common_gruen,lag, social_media_usage_gruen)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - BÜNDNIS 90/ DIE GRÜNEN")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_gruen.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.439438
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  384
Model:                          Logit   Df Residuals:                      374
Method:                           MLE   Df Model:                            9
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.02469
Time:                        18:29:06   Log-Likelihood:                -168.74
converged:                       True   LL-Null:                       -173.02
Covariance Type:            nonrobust   LLR p-value:                    0.4805
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -1.8715      0.423     -4.429      0.000      -2.

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Optimization terminated successfully.
         Current function value: 0.455196
         Iterations 6


c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  354
Model:                          Logit   Df Residuals:                      344
Method:                           MLE   Df Model:                            9
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                0.009454
Time:                        18:29:06   Log-Likelihood:                -161.14
converged:                       True   LL-Null:                       -162.68
Covariance Type:            nonrobust   LLR p-value:                    0.9612
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -1.2674      0.443     -2.858      0.004      -2.137      -0.398
issue attention Facebook     -0.3643      0.998     -0.365      0.715      -2.320     

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [572]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control_test(rede_reduced_gruen, post_reduced_gruen, post_common_gruen,lag, social_media_usage_gruen, rede_komplex_gruen, posts_komplex_gruen)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - BÜNDNIS 90/ DIE GRÜNEN")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_gruen_complex.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.439180
         Iterations 6


                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  384
Model:                          Logit   Df Residuals:                      372
Method:                           MLE   Df Model:                           11
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.02526
Time:                        18:29:06   Log-Likelihood:                -168.65
converged:                       True   LL-Null:                       -173.02
Covariance Type:            nonrobust   LLR p-value:                    0.6458
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -1.6912      0.684     -2.474      0.013      -3.031      -0.351
issue attention Facebook     -0.1240      0.940     -0.132      0.895      -1.967     

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Optimization terminated successfully.
         Current function value: 0.440610
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  366
Model:                          Logit   Df Residuals:                      354
Method:                           MLE   Df Model:                           11
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.03148
Time:                        18:29:06   Log-Likelihood:                -161.26
converged:                       True   LL-Null:                       -166.51
Covariance Type:            nonrobust   LLR p-value:                    0.4875
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.1168      0.752     -2.816      0.005      -3.

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Präzision: 0.0
Recall: 0.0
F1-Score: 0.0
Brier-Score: 0.1361789630791754
Confusion-Matrix:
[[304   0]
 [ 62   0]]
Optimization terminated successfully.
         Current function value: 0.435410
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  360
Model:                          Logit   Df Residuals:                      348
Method:                           MLE   Df Model:                           11
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.05220
Time:                        18:29:06   Log-Likelihood:                -156.75
converged:                       True   LL-Null:                       -165.38
Covariance Type:            nonrobust   LLR p-value:                    0.1002
                                coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Die Linke

In [47]:
subset_posts_linke = subset_posts[subset_posts["partei"] == "DIE LINKE."]
subset_reden_linke = subset_reden[subset_reden["partei"] == "DIE LINKE."]
social_media_usage_linke = subset_posts_linke.groupby('date').size()
subset_reden_linke["date"] = pd.to_datetime(subset_reden_linke["date"])
subset_posts_linke["date"] = pd.to_datetime(subset_posts_linke["date"])
reden_komplexität_täglich_linke = subset_reden_linke.groupby('date')['komplexität'].sum()
posts_komplexität_täglich_linke = subset_posts_linke.groupby('date')['komplexität'].sum()
# 🔹 2️⃣ Gruppiere die Themen täglich und zähle die Häufigkeit der Themen pro Tag
redethemen_täglich_linke = subset_reden_linke.groupby(['date', 'Topic']).size().unstack(fill_value=0)
postthemen_täglich_linke = subset_posts_linke.groupby(['date', 'Topic']).size().unstack(fill_value=0)
# Führe einen inner join durch, damit nur gemeinsame Tage beibehalten werden
common_dates_linke = redethemen_täglich_linke.index.intersection(postthemen_täglich_linke.index)
redethemen_täglich_aligned_linke = redethemen_täglich_linke.loc[common_dates_linke]
postthemen_täglich_aligned_linke = postthemen_täglich_linke.loc[common_dates_linke]
rede_komplex_linke = reden_komplexität_täglich_linke.loc[common_dates_linke]
posts_komplex_linke = posts_komplexität_täglich_linke.loc[common_dates_linke]

C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\3515888887.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_reden_linke["date"] = pd.to_datetime(subset_reden_linke["date"])
C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\3515888887.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_posts_linke["date"] = pd.to_datetime(subset_posts_linke["date"])


In [574]:
posts_komplex_linke

date
2022-05-31    119
2022-06-01    191
2022-06-02    127
2022-06-03    123
2022-06-22     92
             ... 
2023-04-27     86
2023-04-28     42
2023-05-10     71
2023-05-11    127
2023-05-12    285
Name: komplexität, Length: 61, dtype: int64

In [48]:
rede_common_linke, post_common_linke = filter_common_topics(redethemen_täglich_aligned_linke, postthemen_täglich_aligned_linke)
# Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
reden_relativ_linke =  rede_common_linke.div(rede_common_linke.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
post_relativ_linke = post_common_linke.div(post_common_linke.sum(axis=1), axis=0)
reden_relativ_linke_red = remove_near_constant(reden_relativ_linke)
post_relativ_linke_red = remove_near_constant(post_relativ_linke)
rede_reduced_linke, post_reduced_linke = filter_common_topics(reden_relativ_linke_red, post_relativ_linke_red)

Gemeinsame Topics: {0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 28, 29, 30, 34, 35, 36, 40, 43, 52, 53, 54, 55, 56, 58, 64, 68, 74, 75, 79, 85, 92}
Gemeinsame Topics: {0, 1, 2, 6, 8}


In [576]:
post_reduced_linke

Topic,0,1,2,6,8
date,,,,,
2022-05-31,0.000000,0.000000,0.500000,0.0,0.000000
2022-06-01,0.000000,0.000000,0.000000,0.5,0.250000
2022-06-02,0.000000,0.333333,0.666667,0.0,0.000000
2022-06-03,0.000000,0.200000,0.600000,0.0,0.000000
2022-06-22,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...
2023-04-27,1.000000,0.000000,0.000000,0.0,0.000000
2023-04-28,0.000000,0.000000,0.000000,0.0,0.000000
2023-05-10,0.000000,0.000000,0.000000,0.0,0.500000


In [577]:
logistic_regression_with_lags_SM(rede_reduced_linke,post_reduced_linke, post_common_linke)

Optimization terminated successfully.
         Current function value: 0.393246
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  295
Model:                          Logit   Df Residuals:                      292
Method:                           MLE   Df Model:                            2
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                0.009163
Time:                        18:29:06   Log-Likelihood:                -116.01
converged:                       True   LL-Null:                       -117.08
Covariance Type:            nonrobust   LLR p-value:                    0.3421
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.8982      0.198     -9.591      0.000      -2.286      -1.510
posts_relative   

In [578]:
logistic_regression_with_lags_SM_fixed_effects(rede_reduced_linke,post_reduced_linke, post_common_linke)

Optimization terminated successfully.
         Current function value: 0.387441
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  295
Model:                          Logit   Df Residuals:                      288
Method:                           MLE   Df Model:                            6
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.02379
Time:                        18:29:06   Log-Likelihood:                -114.29
converged:                       True   LL-Null:                       -117.08
Covariance Type:            nonrobust   LLR p-value:                    0.4729
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.0545      0.417     -4.921      0.000      -2.873      -1.236
posts_relative   

In [579]:
# Schleife für logistische Regression mit Lags von -7 bis 7
for n in range(1, 8):  # Einschließlich 1 bis 7
    try:
        print(f"\nRegression für Lag {n}:")
        log_reg_FE_control(rede_reduced_linke, post_reduced_linke, post_common_linke,n, social_media_usage_linke)
    except Exception as e:
        print(f"Fehler bei Lag {n}: {e}")


Regression für Lag 1:
Optimization terminated successfully.
         Current function value: 0.379926
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  295
Model:                          Logit   Df Residuals:                      286
Method:                           MLE   Df Model:                            8
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.04272
Time:                        18:29:06   Log-Likelihood:                -112.08
converged:                       True   LL-Null:                       -117.08
Covariance Type:            nonrobust   LLR p-value:                    0.2647
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.7960      0.581     -4.

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anac


Recall: 0.0
F1-Score: 0.0
Brier-Score: 0.11370889701446807
Confusion-Matrix:
[[242   0]
 [ 38   0]]

Regression für Lag 5:
Optimization terminated successfully.
         Current function value: 0.378094
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  275
Model:                          Logit   Df Residuals:                      266
Method:                           MLE   Df Model:                            8
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.04265
Time:                        18:29:07   Log-Likelihood:                -103.98
converged:                       True   LL-Null:                       -108.61
Covariance Type:            nonrobust   LLR p-value:                    0.3206
                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [580]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control(rede_reduced_linke, post_reduced_linke, post_common_linke,lag, social_media_usage_linke)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - DIE LINKE")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_linke.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.379926
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  295
Model:                          Logit   Df Residuals:                      286
Method:                           MLE   Df Model:                            8
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.04272
Time:                        18:29:07   Log-Likelihood:                -112.08
converged:                       True   LL-Null:                       -117.08
Covariance Type:            nonrobust   LLR p-value:                    0.2647
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.7960      0.581     -4.817      0.000      -3.

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anac

Präzision: 0.0
Recall: 0.0
F1-Score: 0.0
Brier-Score: 0.1124761159226722
Confusion-Matrix:
[[238   0]
 [ 37   0]]
Optimization terminated successfully.
         Current function value: 0.374665
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      261
Method:                           MLE   Df Model:                            8
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.04586
Time:                        18:29:07   Log-Likelihood:                -101.16
converged:                       True   LL-Null:                       -106.02
Covariance Type:            nonrobust   LLR p-value:                    0.2848
                                coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [581]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7 
    models.append(model)
    model, auc_roc, f1 = log_reg_FE_control_test(rede_reduced_linke, post_reduced_linke, post_common_linke,lag,social_media_usage_linke, rede_komplex_linke, posts_komplex_linke)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - DIE LINKE")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_linke_complex.html", "w") as f:
    f.write(stargazer.render_html())

ValueError: Length mismatch: Expected axis has 300 elements, new values have 295 elements

#### FDP

In [49]:
subset_posts_fdp = subset_posts[subset_posts["partei"] == "FDP"]
subset_reden_fdp = subset_reden[subset_reden["partei"] == "FDP"]
social_media_usage_fdp = subset_posts_fdp.groupby('date').size()
subset_reden_fdp["date"] = pd.to_datetime(subset_reden_fdp["date"])
subset_posts_fdp["date"] = pd.to_datetime(subset_posts_fdp["date"])
reden_komplexität_täglich_fdp = subset_reden_fdp.groupby('date')['komplexität'].sum()
posts_komplexität_täglich_fdp = subset_posts_fdp.groupby('date')['komplexität'].sum()
# 🔹 2️⃣ Gruppiere die Themen täglich und zähle die Häufigkeit der Themen pro Tag
redethemen_täglich_fdp = subset_reden_fdp.groupby(['date', 'Topic']).size().unstack(fill_value=0)
postthemen_täglich_fdp = subset_posts_fdp.groupby(['date', 'Topic']).size().unstack(fill_value=0)
# Führe einen inner join durch, damit nur gemeinsame Tage beibehalten werden
common_dates_fdp = redethemen_täglich_fdp.index.intersection(postthemen_täglich_fdp.index)
redethemen_täglich_aligned_fdp = redethemen_täglich_fdp.loc[common_dates_fdp]
postthemen_täglich_aligned_fdp = postthemen_täglich_fdp.loc[common_dates_fdp]
rede_komplex_fdp = reden_komplexität_täglich_fdp.loc[common_dates_fdp]
posts_komplex_fdp = posts_komplexität_täglich_fdp.loc[common_dates_fdp]

C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\1557700233.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_reden_fdp["date"] = pd.to_datetime(subset_reden_fdp["date"])
C:\Users\pturl\AppData\Local\Temp\ipykernel_13360\1557700233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_posts_fdp["date"] = pd.to_datetime(subset_posts_fdp["date"])


In [50]:
rede_common_fdp, post_common_fdp = filter_common_topics(redethemen_täglich_aligned_fdp, postthemen_täglich_aligned_fdp)
# Berechnung der relativen Anteile jedes Themas an allen Themen des jeweiligen Tages
reden_relativ_fdp =  rede_common_fdp.div(rede_common_fdp.sum(axis=1), axis=0)  # Zeilenweise Division durch Gesamtsumme
post_relativ_fdp = post_common_fdp.div(post_common_fdp.sum(axis=1), axis=0)
reden_relativ_fdp_red = remove_near_constant(reden_relativ_fdp)
post_relativ_fdp_red = remove_near_constant(post_relativ_fdp)
rede_reduced_fdp, post_reduced_fdp = filter_common_topics(reden_relativ_fdp_red, post_relativ_fdp_red)
fdp_common_topics = post_common_fdp.loc[:, rede_reduced_fdp.columns]

Gemeinsame Topics: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 39, 40, 41, 43, 44, 45, 46, 52, 53, 54, 56, 58, 59, 60, 61, 63, 64, 65, 69, 70, 72, 74, 75, 77, 78, 79, 80, 83, 90, 92}
Gemeinsame Topics: {0, 1, 2, 3, 4, 34, 6, 7, 8, 10, 43, 14, 15, 22, 23}


In [ ]:
post_common_fdp

Topic,0,1,2,3,4,5,6,7,8,9,...,72,74,75,77,78,79,80,83,90,92
date,,,,,,,,,,,,,,,,,,,,,
2022-05-31,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-06-01,0,7,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-06-02,1,2,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
2022-06-03,0,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
2022-06-22,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-27,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-28,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2023-05-10,0,2,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [445]:
logistic_regression_with_lags_SM(rede_reduced_fdp,post_reduced_fdp, post_common_fdp)

Optimization terminated successfully.
         Current function value: 0.523590
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  975
Model:                          Logit   Df Residuals:                      972
Method:                           MLE   Df Model:                            2
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.02628
Time:                        17:49:53   Log-Likelihood:                -510.50
converged:                       True   LL-Null:                       -524.28
Covariance Type:            nonrobust   LLR p-value:                 1.037e-06
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.4418      0.092    -15.691      0.000      -1.622      -1.262
posts_relative   

In [446]:
logistic_regression_with_lags_SM_fixed_effects(rede_reduced_fdp,post_reduced_fdp, fdp_common_topics)

Optimization terminated successfully.
         Current function value: 0.489450
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  975
Model:                          Logit   Df Residuals:                      958
Method:                           MLE   Df Model:                           16
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.08977
Time:                        17:49:55   Log-Likelihood:                -477.21
converged:                       True   LL-Null:                       -524.28
Covariance Type:            nonrobust   LLR p-value:                 4.306e-13
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.7549      0.271     -2.787      0.005      -1.286      -0.224
posts_relative   

In [447]:
# Schleife für logistische Regression mit Lags von -7 bis 7
for n in range(1, 8):  # Einschließlich 1 bis 7
    try:
        print(f"\nRegression für Lag {n}:")
        log_reg_FE_control(rede_reduced_fdp, post_reduced_fdp, post_common_fdp,n, social_media_usage_fdp)
    except Exception as e:
        print(f"Fehler bei Lag {n}: {e}")


Regression für Lag 1:
Optimization terminated successfully.
         Current function value: 0.486901
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  975
Model:                          Logit   Df Residuals:                      956
Method:                           MLE   Df Model:                           18
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.09451
Time:                        17:49:57   Log-Likelihood:                -474.73
converged:                       True   LL-Null:                       -524.28
Covariance Type:            nonrobust   LLR p-value:                 3.235e-13
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.7349      0.338     -2.

In [584]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control(rede_reduced_fdp, post_reduced_fdp, post_common_fdp,lag, social_media_usage_fdp)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - FDP")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_fdp.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.486901
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  975
Model:                          Logit   Df Residuals:                      956
Method:                           MLE   Df Model:                           18
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.09451
Time:                        18:29:24   Log-Likelihood:                -474.73
converged:                       True   LL-Null:                       -524.28
Covariance Type:            nonrobust   LLR p-value:                 3.235e-13
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.7349      0.338     -2.172      0.030      -1.

In [585]:
models = []
metrics = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    model, auc_roc, f1 = log_reg_FE_control_test(rede_reduced_fdp, post_reduced_fdp, post_common_fdp,lag, social_media_usage_fdp, rede_komplex_fdp, posts_komplex_fdp)
    models.append(model)
    metrics.append((auc_roc, f1))

# Stargazer-Tabelle erstellen
stargazer = Stargazer(models)
stargazer.title("Vergleich mit Lags 1-7")
stargazer.custom_columns([f"Lag {i}" for i in range(1, 8)], [1] * 7)
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung auf Social Media - FDP")

# Dynamisch AUC-ROC und F1-Score hinzufügen
custom_notes = [f"Lag {i}: AUC-ROC = {metrics[i-1][0]:.3f}, F1-Score = {metrics[i-1][1]:.3f}" for i in range(1, 8)]
stargazer.add_custom_notes(custom_notes)

# Exportieren
with open("regression_table_comparison_fdp_complex.html", "w") as f:
    f.write(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.486164
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  975
Model:                          Logit   Df Residuals:                      954
Method:                           MLE   Df Model:                           20
Date:                Tue, 28 Jan 2025   Pseudo R-squ.:                 0.09588
Time:                        18:29:27   Log-Likelihood:                -474.01
converged:                       True   LL-Null:                       -524.28
Covariance Type:            nonrobust   LLR p-value:                 1.009e-12
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.5522      0.511     -1.082      0.279      -1.

#### Vergleiche der beiden Modelle mit unterschiedlichen Zielvariablen über die Lags hinweg


In [598]:
from stargazer.stargazer import Stargazer

# Initialisiere eine Liste für Stargazer-Tabellen
stargazer_tables = []
partei= "spd"
# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    # Modell 1: Zielvariable basiert auf rede_common_fdp
    model1, auc_roc1, f1_1 = log_reg_FE_control_test(
        rede_reduced_{partei}, post_reduced_spd, rede_common_spd, lag, social_media_usage_spd, rede_komplex_spd, posts_komplex_spd
    )
    
    # Modell 2: Zielvariable basiert auf post_common_fdp
    model2, auc_roc2, f1_2 = log_reg_FE_control_test(
        rede_reduced_spd, post_reduced_spd, post_common_spd, lag, social_media_usage_fdp, rede_komplex_fdp, posts_komplex_fdp
    )
    
    # Stargazer-Tabelle für den Vergleich der beiden Modelle
    stargazer = Stargazer([model1, model2])
    stargazer.title(f" FDP Vergleich für Lag {lag}")
    stargazer.custom_columns(
        [f"Bundestag ", 
         f"Facebook"], 
        [1, 1]
    )
    stargazer.significant_digits(3)
    stargazer.dependent_variable_name("Themenerwähnung FDP")
    
    # AUC-ROC und F1-Score für beide Modelle als individuelle Notizen
    custom_notes = [
        f"Modell 1 (Reden als Zielvariable): AUC-ROC = {auc_roc1:.3f}, F1-Score = {f1_1:.3f}",
        f"Modell 2 (Posts als Zielvariable): AUC-ROC = {auc_roc2:.3f}, F1-Score = {f1_2:.3f}"
    ]
    stargazer.add_custom_notes(custom_notes)
    
    # Exportiere jede Tabelle separat
    filename = f"regression tables/regression_table_comparison_fdp_lag_{lag}.html"
    with open(filename, "w") as f:
        f.write(stargazer.render_html())
    
    # Speichere die Tabelle in der Liste
    stargazer_tables.append((lag, stargazer))


Optimization terminated successfully.
         Current function value: 0.534749
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  975
Model:                          Logit   Df Residuals:                      954
Method:                           MLE   Df Model:                           20
Date:                Wed, 29 Jan 2025   Pseudo R-squ.:                  0.1025
Time:                        10:58:40   Log-Likelihood:                -521.38
converged:                       True   LL-Null:                       -580.94
Covariance Type:            nonrobust   LLR p-value:                 4.149e-16
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.9291      0.473     -1.963      0.050      -1.

In [610]:
for lag in range(1,8):
    model1,auc_roc1,f1_1=log_reg_FE_control_test(rede_relativ_reduced, post_relativ_reduced, rede_common,lag, social_media_usage,rede_komplex, posts_komplex)
    model2,auc_roc2,f1_2=log_reg_FE_control_test(rede_relativ_reduced, post_relativ_reduced, post_common,lag, social_media_usage,rede_komplex, posts_komplex)
     # Stargazer-Tabelle für den Vergleich der beiden Modelle
    stargazer = Stargazer([model1, model2])
    stargazer.title(f" Aggregiert Vergleich für Lag {lag}")
    stargazer.custom_columns(
        [f"Bundestag ", 
         f"Facebook"], 
        [1, 1]
    )
    stargazer.significant_digits(3)
    stargazer.dependent_variable_name("Themenerwähnung Aggregiert")
    
    # AUC-ROC und F1-Score für beide Modelle als individuelle Notizen
    custom_notes = [
        f"Modell 1 (Reden als Zielvariable): AUC-ROC = {auc_roc1:.3f}, F1-Score = {f1_1:.3f}",
        f"Modell 2 (Posts als Zielvariable): AUC-ROC = {auc_roc2:.3f}, F1-Score = {f1_2:.3f}"
    ]
    stargazer.add_custom_notes(custom_notes)
    
    # Exportiere jede Tabelle separat
    filename = f"regression tables/regression_table_comparison_agg_lag_{lag}.html"
    with open(filename, "w") as f:
        f.write(stargazer.render_html())
    
    # Speichere die Tabelle in der Liste
    stargazer_tables.append((lag, stargazer))


Optimization terminated successfully.
         Current function value: 0.539528
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2234
Method:                           MLE   Df Model:                           40
Date:                Wed, 29 Jan 2025   Pseudo R-squ.:                  0.1434
Time:                        11:45:54   Log-Likelihood:                -1227.4
converged:                       True   LL-Null:                       -1432.8
Covariance Type:            nonrobust   LLR p-value:                 4.902e-63
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.0505      0.524     -0.096      0.923      -1.

In [57]:
# Mastertabelle aggregierte Daten
# Liste zur Speicherung der Modelle und der zugehörigen Lag-Beschriftungen
models = []
lag_labels = []

# Schleife über Lags
for lag in range(1, 8):
    model1, auc_roc1, f1_1 = log_reg_FE_control_test(
        rede_relativ_reduced, post_relativ_reduced, rede_common, lag, 
        social_media_usage, rede_komplex, posts_komplex
    )
    
    model2, auc_roc2, f1_2 = log_reg_FE_control_test(
        rede_relativ_reduced, post_relativ_reduced, post_common, lag, 
        social_media_usage, rede_komplex, posts_komplex
    )

    # Modelle und Lag-Beschriftung speichern
    models.extend([model1, model2])
    lag_labels.extend([f"Lag {lag} - Bundestag", f"Lag {lag} - Facebook"])

# Eine einzige Stargazer-Tabelle mit allen Lags erstellen
stargazer = Stargazer(models)
stargazer.title("Aggregierter Vergleich für alle Lags")
stargazer.custom_columns(lag_labels, [1] * len(lag_labels))
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung Aggregiert")

# AUC-ROC und F1-Score für jedes Lag als Notizen hinzufügen
custom_notes = [
    f"Lag {lag}: Modell 1 (Bundestag) - AUC-ROC = {auc_roc1:.3f}, F1-Score = {f1_1:.3f}; "
    f"Modell 2 (Facebook) - AUC-ROC = {auc_roc2:.3f}, F1-Score = {f1_2:.3f}"
    for lag in range(1, 8)
]
stargazer.add_custom_notes(custom_notes)

# HTML-Datei speichern
filename = "regression tables/regression_table_comparison_agg_all_lags.html"
with open(filename, "w") as f:
    f.write(stargazer.render_html())


Optimization terminated successfully.
         Current function value: 0.539528
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2275
Model:                          Logit   Df Residuals:                     2234
Method:                           MLE   Df Model:                           40
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.1434
Time:                        11:10:00   Log-Likelihood:                -1227.4
converged:                       True   LL-Null:                       -1432.8
Covariance Type:            nonrobust   LLR p-value:                 4.902e-63
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.0505      0.524     -0.096      0.923      -1.

In [ ]:
# Partei einmal definieren
partei = "gruen"

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    # Dynamische Referenzierung der Variablen basierend auf dem Parteinamen
    model1, auc_roc1, f1_1 = log_reg_FE_control_test(
        globals()[f"rede_reduced_{partei}"], 
        globals()[f"post_reduced_{partei}"], 
        globals()[f"rede_common_{partei}"], 
        lag, 
        globals()[f"social_media_usage_{partei}"], 
        globals()[f"rede_komplex_{partei}"], 
        globals()[f"posts_komplex_{partei}"]
    )

    model2, auc_roc2, f1_2 = log_reg_FE_control_test(
        globals()[f"rede_reduced_{partei}"], 
        globals()[f"post_reduced_{partei}"], 
        globals()[f"post_common_{partei}"], 
        lag, 
        globals()[f"social_media_usage_{partei}"], 
        globals()[f"rede_komplex_{partei}"], 
        globals()[f"posts_komplex_{partei}"]
    )

    # Stargazer-Tabelle für den Vergleich der beiden Modelle
    stargazer = Stargazer([model1, model2])
    stargazer.title(f"Grüne Vergleich für Lag {lag} - {partei.upper()}")
    stargazer.custom_columns(
        [f"Bundestag", 
         f"Facebook"], 
        [1, 1]
    )
    stargazer.significant_digits(3)
    stargazer.dependent_variable_name("Themenerwähnung Grüne")

    # AUC-ROC und F1-Score als Notizen
    custom_notes = [
        f"Modell 1 (Reden als Zielvariable): AUC-ROC = {auc_roc1:.3f}, F1-Score = {f1_1:.3f}",
        f"Modell 2 (Posts als Zielvariable): AUC-ROC = {auc_roc2:.3f}, F1-Score = {f1_2:.3f}"
    ]
    stargazer.add_custom_notes(custom_notes)

    # HTML-Datei mit größerem Spaltenabstand speichern
    filename = f"regression tables/regression_table_comparison_{partei}_lag_{lag}.html"
    with open(filename, "w") as f:
        f.write(stargazer.render_html())


Optimization terminated successfully.
         Current function value: 0.608104
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  295
Model:                          Logit   Df Residuals:                      286
Method:                           MLE   Df Model:                            8
Date:                Wed, 29 Jan 2025   Pseudo R-squ.:                 0.03229
Time:                        11:37:50   Log-Likelihood:                -179.39
converged:                       True   LL-Null:                       -185.38
Covariance Type:            nonrobust   LLR p-value:                    0.1525
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.4501      0.392     -1.148      0.251      -1.

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Präzision: 0.0
Recall: 0.0
F1-Score: 0.0
Brier-Score: 0.11344598916471275
Confusion-Matrix:
[[247   0]
 [ 38   0]]
Optimization terminated successfully.
         Current function value: 0.613708
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  280
Model:                          Logit   Df Residuals:                      271
Method:                           MLE   Df Model:                            8
Date:                Wed, 29 Jan 2025   Pseudo R-squ.:                 0.02676
Time:                        11:37:50   Log-Likelihood:                -171.84
converged:                       True   LL-Null:                       -176.56
Covariance Type:            nonrobust   LLR p-value:                    0.3059
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Optimization terminated successfully.
         Current function value: 0.374665
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      261
Method:                           MLE   Df Model:                            8
Date:                Wed, 29 Jan 2025   Pseudo R-squ.:                 0.04586
Time:                        11:37:50   Log-Likelihood:                -101.16
converged:                       True   LL-Null:                       -106.02
Covariance Type:            nonrobust   LLR p-value:                    0.2848
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.9317      0.654     -4.482      0.000      -4.

c:\Users\pturl\anaconda3_new\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [56]:
## Mastertabelle mit allen Lags in einer Tabelle (um Interpretation leichter zu machen)
# Partei einmal definieren
partei = "linke"

# Liste zur Speicherung der Modelle
models = []
lag_labels = []

# Schleife über Lags
for lag in range(1, 8):  # Für Lags 1 bis 7
    # Dynamische Referenzierung der Variablen basierend auf dem Parteinamen
    model1, auc_roc1, f1_1 = log_reg_FE_control_test(
        globals()[f"rede_reduced_{partei}"], 
        globals()[f"post_reduced_{partei}"], 
        globals()[f"rede_common_{partei}"], 
        lag, 
        globals()[f"social_media_usage_{partei}"], 
        globals()[f"rede_komplex_{partei}"], 
        globals()[f"posts_komplex_{partei}"]
    )

    model2, auc_roc2, f1_2 = log_reg_FE_control_test(
        globals()[f"rede_reduced_{partei}"], 
        globals()[f"post_reduced_{partei}"], 
        globals()[f"post_common_{partei}"], 
        lag, 
        globals()[f"social_media_usage_{partei}"], 
        globals()[f"rede_komplex_{partei}"], 
        globals()[f"posts_komplex_{partei}"]
    )

    # Modelle und Lag-Beschriftung speichern
    models.extend([model1, model2])
    lag_labels.extend([f"Lag {lag} - Bundestag", f"Lag {lag} - Facebook"])

# Eine einzige Stargazer-Tabelle mit allen Lags erstellen
stargazer = Stargazer(models)
stargazer.title(f"Vergleich aller Lags für {partei.upper()}")
stargazer.custom_columns(lag_labels, [1] * len(lag_labels))
stargazer.significant_digits(3)
stargazer.dependent_variable_name("Themenerwähnung Grüne")

# AUC-ROC und F1-Score für jedes Lag als Notizen hinzufügen
custom_notes = [
    f"Lag {lag}: Modell 1 (Bundestag) - AUC-ROC = {auc_roc1:.3f}, F1-Score = {f1_1:.3f}; "
    f"Modell 2 (Facebook) - AUC-ROC = {auc_roc2:.3f}, F1-Score = {f1_2:.3f}"
    for lag in range(1, 8)
]
stargazer.add_custom_notes(custom_notes)

# HTML-Datei speichern
filename = f"regression tables/regression_table_comparison_{partei}_all_lags.html"
with open(filename, "w") as f:
    f.write(stargazer.render_html())


ValueError: Length mismatch: Expected axis has 300 elements, new values have 295 elements